# Read Robinhood Statement .pdf

In [2]:
import pdfplumber
import pandas as pd
import re

def extract_robinhood_portfolio(pdf_path, output_excel_path):
    """
    Extracts portfolio data and ensures tickers are Yahoo Finance compatible 
    (e.g., changing BRK.B to BRK-B).

    Arguments: Robinhood's Statement pdf path and output excel path

    Returns: excel file with  | Ticker | Current QT | Price | Mkt Value | Est Dividend | Yield | Weight |

    """
    # Regex pattern to capture: Ticker, Qty, Price, Mkt Value, Dividend, Weight
    # Supports tickers with dots like BRK.B
    portfolio_pattern = re.compile(
        r'([A-Z0-9.]{1,6})\s+Margin\s+([\d,.]+)\s+\$([\d,.]+)\s+([\d,.]+\$|\$[\d,.]+)\s+\$([\d,.]+)\s+([\d,.]+\%)'
    )

    extracted_data = []

    with pdfplumber.open(pdf_path) as pdf:
        print(f"Scanning {len(pdf.pages)} pages...")
        for page in pdf.pages:
            text = page.extract_text()
            if not text:
                continue
            
            matches = portfolio_pattern.findall(text)
            for m in matches:
                ticker = m[0].strip()
                
                # --- YFINANCE COMPATIBILITY CHECK ---
                # Change dots to hyphens for Yahoo Finance compatibility
                # This fixes BRK.B -> BRK-B and others like BF.B -> BF-B
                if "." in ticker:
                    ticker = ticker.replace(".", "-")

                extracted_data.append({
                    "Ticker": ticker,
                    "Current QTY": m[1],
                    "Price": m[2],
                    "Mkt Value": m[3].replace('$', ''),
                    "Est Dividend Yield": m[4],
                    "Weight": m[5]
                })

    if not extracted_data:
        print("No portfolio data found.")
        return None

    df = pd.DataFrame(extracted_data)

    # --- Data Cleaning ---
    def clean_numeric(val):
        if isinstance(val, str):
            # Remove symbols and handle cases where $ is at the end (e.g. 123.04$)
            cleaned = val.replace(',', '').replace('$', '').replace('%', '').strip()
            try:
                return float(cleaned)
            except ValueError:
                return 0.0
        return val

    df["Current QTY"] = df["Current QTY"].apply(clean_numeric)
    df["Price"] = df["Price"].apply(clean_numeric)
    df["Mkt Value"] = df["Mkt Value"].apply(clean_numeric)
    df["Est Dividend Yield"] = df["Est Dividend Yield"].apply(clean_numeric)
    df["Weight"] = df["Weight"].apply(lambda x: clean_numeric(x) / 100)

    # Remove duplicates
    df = df.drop_duplicates(subset=['Ticker', 'Current QTY'])

    df.to_excel(output_excel_path, index=False)
    print(f"Successfully saved {len(df)} rows to {output_excel_path}")
    print(f"{len(df)} Tickers")
    print(f"Mkt Value: ${df['Mkt Value'].sum():,.2f}")
    print(f"Weight: {df['Weight'].sum():,.2%} (does not include Crypto)")
    return df


# Fundamental Analysis

## ROIC - Return on Invested Capital

ROIC measures how efficiently a company uses its invested capital to generate profits

In [ ]:
import yfinance as yf

def get_roic(ticker_symbol: str):
    """
    Calculates the Return on Invested Capital (ROIC) for a given stock ticker.

    ROIC = NOPAT / Invested Capital
    Where:
    - NOPAT = EBIT * (1 - Tax Rate)
    - Invested Capital = Total Equity + Total Debt - Cash & Cash Equivalents

    Args:
        ticker_symbol (str): The stock ticker symbol (e.g., 'AAPL').

    Returns:
        float: The ROIC as a percentage, or None if data is not available.
    """
    try:
        stock = yf.Ticker(ticker_symbol)
        
        # Fetch financial statements (income statement and balance sheet)
        income_stmt = stock.income_stmt.T
        balance_sheet = stock.balance_sheet.T

        # Get the latest data point from each financial statement
        latest_income_stmt = income_stmt.iloc[0]
        latest_balance_sheet = balance_sheet.iloc[0]

        # --- Calculate NOPAT (Net Operating Profit After Tax) ---
        ebit = latest_income_stmt['EBIT']

        
        # Calculate the effective tax rate
        tax_rate = latest_income_stmt['Tax Rate For Calcs']
        nopat = ebit * (1 - tax_rate)

        # --- Calculate Invested Capital ---
        total_equity = latest_balance_sheet['Stockholders Equity']
        total_debt = latest_balance_sheet['Total Debt']
        cash = latest_balance_sheet['Cash And Cash Equivalents']

        invested_capital = total_equity + total_debt - cash

        # --- Calculate ROIC ---
        roic = (nopat / invested_capital) * 100
        return round(roic, 2)

    except (KeyError, IndexError):
        print(f"Error (get_roic): Financial data for '{ticker_symbol}' not available or incomplete.")
        return 'N/A'
    except Exception as e:
        print(f"(get_roic) An unexpected error occurred for '{ticker_symbol}': {e}")
        return 'N/A'


# # --- Example Usage ---
# # Get the annual ROIC for Apple
# aapl_roic = get_roic('QYLD')
# if aapl_roic is not None:
#     print(f"Annual ROIC for VOOV: {aapl_roic}%")
#     print(aapl_roic)
    
# # Example of a ticker with potentially missing data
# bad_ticker_roic = get_roic('INVALIDTICKER')



'\n# --- Example Usage ---\n\n# Get the annual ROIC for Apple\naapl_roic = get_roic(\'QYLD\')\nif aapl_roic is not None:\n    print(f"Annual ROIC for VOOV: {aapl_roic}%")\n    print(aapl_roic)\n\n# Example of a ticker with potentially missing data\nbad_ticker_roic = get_roic(\'INVALIDTICKER\')\n'

## EBIT Margin

In [ ]:
import yfinance as yf

def get_ebit_margin(ticker_symbol: str):
    """
    Fetches financial data from yfinance and calculates the latest annual EBIT margin.

    Args:
        ticker_symbol (str): The stock ticker symbol (e.g., 'AAPL').

    Returns:
        float: The EBIT margin as a percentage, or None if data is not available.
    """
    try:
        # Create a Ticker object
        stock = yf.Ticker(ticker_symbol)

        # Get the annual income statement
        income_stmt = stock.income_stmt.T

        # Check if the income statement DataFrame is empty
        if income_stmt.empty:
            print(f"(get_ebit_margin) No annual income statement found for '{ticker_symbol}'.")
            return None

        # Get the last Income Statement
        last_income_stmt = income_stmt.iloc[0]

        # Get the most recent annual EBIT and Total Revenue
        ebit = last_income_stmt['EBIT']
        total_revenue = last_income_stmt['Total Revenue']

        # Calculate the EBIT margin as a percentage
        ebit_margin = (ebit / total_revenue) * 100
        
        """
        print(f"--- {ticker_symbol.upper()} ---")
        print(f"Latest Annual EBIT: ${ebit:,.2f}")
        print(f"Latest Annual Total Revenue: ${total_revenue:,.2f}")
        print(f"EBIT Margin: {ebit_margin:.2f}%")
        """
        return round(ebit_margin, 2)

    except ZeroDivisionError:
        print(f"(get_ebit_margin) ZeroDivisionError, Missing or zero financial data for '{ticker_symbol}'.")
        return 'N/A'
    except KeyError:
        print(f"(get_ebit_margin) Could not find 'Earnings Before Interest and Taxes' or 'Total Revenue' for {ticker_symbol}.")
        print("(get_ebit_margin) Please check the Yahoo Finance page for the company's specific financial statement labels.")
        return 'N/A'
    except Exception as e:
        print(f"An error occurred: {e}")
        return 'N/A'
    

# # Example usage for Apple (AAPL) and Microsoft (MSFT)
# a = get_ebit_margin("AAPL")
# print(a)


'\n# Example usage for Apple (AAPL) and Microsoft (MSFT)\na = get_ebit_margin("AAPL")\nprint(a)\n'

## Net Margin

In [ ]:
import yfinance as yf

def get_net_margin(ticker_symbol: str):
    """
    Calculates the net margin for a given stock ticker using annual data from yfinance.

    Args:
        ticker_symbol (str): The stock ticker symbol (e.g., 'AAPL').

    Returns:
        float: The net margin as a percentage, or None if data is not available.
    """
    try:
        stock = yf.Ticker(ticker_symbol)
        income_stmt = stock.income_stmt.T

        # Get the latest Net Income and Total Revenue
        net_income = income_stmt.iloc[0]['Net Income']
        total_revenue = income_stmt.iloc[0]['Total Revenue']

        # Calculate the net margin and round to two decimal places
        net_margin = (net_income / total_revenue) * 100
        return round(net_margin, 2)

    except (KeyError, IndexError):
        print(f"Error (get_net_margin): Financial data for '{ticker_symbol}' not available.")
        return 'N/A'
    except Exception as e:
        print(f"(get_net_margin) An unexpected error occurred for '{ticker_symbol}': {e}")
        return 'N/A'


# # --- Example Usage ---
# # Get the annual net margin for Apple
# aapl_net_margin = get_net_margin('QYLD')
# if aapl_net_margin is not None:
#     print(f"Annual Net Margin for QYLD: {aapl_net_margin}%")
#     print(aapl_net_margin)

# # Example of a ticker with potentially missing or inconsistent data
# bad_ticker_margin = get_net_margin('INVALIDTICKER')


'\n# --- Example Usage ---\n\n# Get the annual net margin for Apple\naapl_net_margin = get_net_margin(\'QYLD\')\nif aapl_net_margin is not None:\n    print(f"Annual Net Margin for QYLD: {aapl_net_margin}%")\n    print(aapl_net_margin)\n\n# Example of a ticker with potentially missing or inconsistent data\nbad_ticker_margin = get_net_margin(\'INVALIDTICKER\')\n'

## CAGR - Compound Annual Growth Rate

In [ ]:
import yfinance as yf
from datetime import datetime

def get_cagr(ticker_symbol: str, start_date: datetime, end_date: datetime):
    """
    Calculates the Compound Annual Growth Rate (CAGR) for a given stock in Percentage %.

    Args:
        ticker_symbol (str): The stock ticker symbol (e.g., 'MSFT').
        start_date (str): The start date in date format.
        end_date (str): The end date in date format.

    Returns:
        float: The CAGR as a percentage, or None if data retrieval fails.
    """
    try:
        stock_data = yf.download(ticker_symbol, start=start_date, end=end_date, auto_adjust=True, progress=False)
        
        beginning_value = stock_data.iloc[0]['Close']
        ending_value = stock_data.iloc[-1]['Close']
        
        # Calculate number of years
        no_days = end_date - start_date
        num_years = no_days.days / 365
        
        if no_days == 0:
            cagr = 0
        else:
            cagr = (ending_value.iloc[0] / beginning_value.iloc[0]) ** (1 / num_years) - 1
        
        return round(cagr * 100, 2)
    
    except (KeyError, IndexError):
        print(f"Error (get_cagr): Data for '{ticker_symbol}' not available for the specified date range.")
        return 'N/A'
    except Exception as e:
        print(f"(get_cagr) An unexpected error occurred: {e}")
        return 'N/A'


# # Example usage of the function
# #Dates
# start_date = datetime(2021, 4, 25) # <- date in which brokerage account started
# today = datetime.today()

# msft_cagr = get_cagr('NVDA', start_date, today)
# if msft_cagr is not None:
#     print(f"Annual CAGR for MSFT: {msft_cagr}%")

'\n# Example usage of the function\n\n#Dates\nstart_date = datetime(2021, 4, 25) # <- date in which brokerage account started\ntoday = datetime.today()\n\nmsft_cagr = get_cagr(\'NVDA\', start_date, today)\nif msft_cagr is not None:\n    print(f"Annual CAGR for MSFT: {msft_cagr}%")\n'

## Get Financial Parameters
(uses some of the above functions)

In [1]:
import yfinance as yf
from datetime import datetime
import pandas as pd

def get_fundamental_analysis(tickers, start_date, end_date, metrics_config:dict):
    """
    Creates a Fundamental Analysis from a list of symbols based on the Financial Ratios defined in "data"
    Arguments: 
        tickers is a list of Symbols in str format
        start_date and end_date in date format,
        config: dictionary with parameter name/Units/Criteria/description,yf_key
    Returns: DataFrame with symbols and Financial Ratios
    """

    all_rows = []
    for ticker_symbol in tickers:
        try:
            t = yf.Ticker(ticker_symbol)
            ticker_info = t.info

            # Check if basic ticker info is available before proceeding
            if not ticker_info or not ticker_info.get('symbol'):
                print(f"Skipping {ticker_symbol}: Ticker info not available.")
                continue
                
            ticker_name = ticker_info['symbol']
            print(f"Processing {ticker_name}")


            # --- PRE-CALCULATE SHARED VARIABLES HERE ---
            fcf = ticker_info.get('freeCashflow', 0)
            mkt_cap = ticker_info.get('marketCap', 0)
            enterprise_value = ticker_info.get('enterpriseValue', 0)

            # -- Profile --
            data = {
                'Name': ticker_info.get('longName', 'N/A'), 
                'Sector': ticker_info.get('sector', 'ETF, others'), 
                'Industry': ticker_info.get('industry', 'ETF, others'),
                'Next Earnings': pd.to_datetime(ticker_info.get('earningsTimestamp', None), unit='s').strftime('%Y-%m-%d') if ticker_info.get('earningsTimestamp', None) else "N/A" # Qualitative, not plotted
                }

            # Automatically pull data based on config
            for metric, meta in metrics_config.items():
                # 1. Try to get direct YFinance value if yf_key exists
                val = ticker_info.get(meta.get('yf_key'), 0) if meta.get('yf_key') else 0
                
                # 2. Derived metrics. Handles specific calculations (when 'yf_key' doesn't exist)
                if metric == 'FCF Yield_%':
                    val = (fcf / mkt_cap)  if (fcf and mkt_cap) else 0.0 # FCF Yield = FCF / Market Cap
                elif metric == 'CAGR_%':
                    val = get_cagr(ticker_name, start_date, end_date) # Uses helper function above
                elif metric == 'EV/FCF':
                    val = (enterprise_value / fcf) if (enterprise_value and fcf > 0) else 0.0 # EV / FCF
                elif metric == 'ROIC_%': 
                    val = get_roic(ticker_name)
                elif metric == 'EBIT Margin_%': 
                    val = get_ebit_margin(ticker_name)
                elif metric == 'Net Margin_%': 
                    val = get_net_margin(ticker_name)
                # elif 'Yield' in metric or '%' in metric or 'Change' in metric:
                #     # Auto-convert yfinance decimals (0.05) to percentages (5.0)
                #     if val and val < 1.0 and val > -1.0: val *= 100

                # Percentage auto-correction (optional but recommended)
                if meta.get('unit') == '%' and 0 < abs(val) < 1.0:
                    val *= 100
                
                data[metric] = val
            
            all_rows.append(pd.Series(data, name=ticker_symbol))
            
        except Exception as e:
            print(f"Error {ticker_symbol}: {e}")

    return pd.DataFrame(all_rows)

## Scorecard - Display Financial Table 

In [ ]:
import pandas as pd
import operator

def display_master_scorecard(fa_df, config):
    """
    Consolidates Fundamental Analysis and Scorecard into a single enriched table.
    - Rows: Score %, Info, Metrics, and Totals.
    - Columns: Unit, Criteria, Tickers..., Description.
    """
    tickers = fa_df.index
    score_data = {}
    
    # Operator mapping for the comparison logic
    ops = {">": operator.gt, "<": operator.lt, ">=": operator.ge, "<=": operator.le}
    
    for ticker in tickers:
        total_passed = 0
        total_failed = 0
        valid_metrics = [m for m in config.keys() if m in fa_df.columns]
        
        ticker_column_display = {}
        
        for metric in valid_metrics:
            val = fa_df.loc[ticker, metric]
            meta = config[metric]
            op_str, threshold = meta['criteria']
            
            # 1. Comparison Logic
            # Check if value is missing (None or NaN)
            if pd.isna(val):
                passed = False
            elif op_str == "between":
                passed = threshold[0] <= val <= threshold[1]
            else:
                # Handle Dynamic EPS (10% of CAGR)
                if 'EPS_usd' in metric and threshold == 10.0:
                    # Check if CAGR is also valid to avoid another error
                    cagr = fa_df.loc[ticker, 'CAGR_%']
                    target = cagr * 0.10 if not pd.isna(cagr) else 0
                    passed = val >= target
                else:
                    passed = ops[op_str](val, threshold)
            
            # 2. Format display: Value + Icon
            if pd.isna(val):
                formatted_val = "NoDataFound"
            else:
                formatted_val = f"{val:.2f}" if isinstance(val, (int, float)) else str(val)
            
            ticker_column_display[metric] = f"{formatted_val} {'✅' if passed else '❌'}"
            
            if passed:
                total_passed += 1
            else:
                total_failed += 1
        
        # 3. Add Totals and Score
        score_pct = (total_passed / len(valid_metrics)) * 100 if valid_metrics else 0
        ticker_column_display['Score %'] = round(score_pct, 2)
        ticker_column_display['Total ✅'] = total_passed
        ticker_column_display['Total ❌'] = total_failed
        
        # 4. Include Profile info
        for info_col in ['Name', 'Sector', 'Industry']:
            if info_col in fa_df.columns:
                ticker_column_display[info_col] = fa_df.loc[ticker, info_col]
        
        score_data[ticker] = ticker_column_display

    # Create DataFrame from calculated results
    master_df = pd.DataFrame(score_data)
    
    # 5. Build Enrichment Columns
    unit_col, criteria_col, desc_col = [], [], []
    for metric_name in master_df.index:
        if metric_name in config:
            item = config[metric_name]
            unit_col.append(item.get('unit', ''))
            criteria_col.append(str(item.get('criteria', '')))
            desc_col.append(item.get('desc', ''))
        else:
            unit_col.append("")
            criteria_col.append("")
            desc_col.append("")

    # 6. Assemble Column Order: [Unit, Criteria, Tickers..., Description]
    master_df.insert(0, "Criteria", criteria_col)
    master_df.insert(0, "Unit", unit_col)
    master_df["Description"] = desc_col # Adds to the end

    # 7. Sort Rows: [Score, Profile, Metrics, Totals]
    top_rows = ['Score %', 'Name', 'Sector', 'Industry']
    bottom_rows = ['Total ✅', 'Total ❌']
    metric_rows = [r for r in master_df.index if r not in top_rows + bottom_rows + ['Description']]
    
    master_df = master_df.reindex(top_rows + metric_rows + bottom_rows)

    # 8. Styling
    # Subset identifies only the Ticker columns for the gradient
    ticker_cols = [col for col in master_df.columns if col not in ['Unit', 'Criteria', 'Description']]
    
    styled_df = master_df.style.background_gradient(
        subset=pd.IndexSlice[['Score %'], ticker_cols], 
        cmap='RdYlGn', 
        axis=1
    ).format(
        # Adds the % symbol to the Score % row for all Ticker columns
        formatter="{:.2f}%", 
        subset=pd.IndexSlice[['Score %'], ticker_cols]
    ).set_table_styles([
        # Align index (Metric names) to the left
        {'selector': 'th.row_heading', 'props': [('text-align', 'left'), ('font-weight', 'bold')]},
        # Center all column headers
        {'selector': 'th.col_heading', 'props': [('text-align', 'center')]}
    ]).set_properties(**{
        'text-align': 'center'
    }, subset=['Unit', 'Criteria'] + ticker_cols) \
     .set_properties(**{'max-width': '500px', 'white-space': 'normal', 'text-align': 'left'},  subset=['Description'])\
     .set_properties(**{'font-weight': 'bold'}, subset=pd.IndexSlice[['Total ✅', 'Total ❌', 'Score %'], :])

#  .set_properties(**{'text-align': 'left'}, subset=['Description']) \



    display(styled_df)
    return master_df

## Bar Plot Financial Ratios

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import operator
from matplotlib.patches import Patch


# Helper for comparison logic
ops = {">": operator.gt, "<": operator.lt, ">=": operator.ge, "<=": operator.le, "==": operator.eq}

def is_between(val, range_tuple):
    return range_tuple[0] <= val <= range_tuple[1]



def plot_ratios(data: pd.DataFrame, config: dict, plots_per_row: int = 4):
    try:
        columns = [k for k in config.keys() if k in data.columns]
        num_plots = len(columns)
        num_rows = int(np.ceil(num_plots / plots_per_row))
    
        fig, axes = plt.subplots(num_rows, plots_per_row, figsize=(plots_per_row * 5, num_rows * 4))
        if num_plots == 1: axes_flat = [axes]
        else: axes_flat = axes.flatten()

        for i, col_name in enumerate(columns):
            conf = config[col_name]
            op_str, threshold = conf['criteria']
            unit = conf.get('unit', '')
            desc = conf.get('desc', '')
            
            # Logic setup
            comparison_func = ops.get(op_str, is_between) if op_str != "between" else is_between

            # Dynamic EPS Logic
            is_dynamic_eps = ('EPS_usd' in col_name and threshold == 10.0 and 'CAGR_%' in data.columns)
            
            bar_colors = []
            row_thresholds = []

            for idx, val in data[col_name].items():
                # --- SAFETY CHECK: for None or NaN values in metrics file i.e. "P/E not found"---
                if pd.isna(val):
                    passed = False
                    bar_colors.append('#D3D3D3') # Neutral Gray for missing data
                    if is_dynamic_eps: 
                        row_thresholds.append(0) # Placeholder
                    continue # Skip to the next bar
                # -----------------------------
                if is_dynamic_eps:
                    # Dynamic Threshold = CAGR * (10/100)
                    cagr = data.loc[idx, 'CAGR_%']
                    # Safety check for CAGR too
                    cagr_val = cagr if not pd.isna(cagr) else 0
                    dyn_thresh = cagr_val * (threshold/100)
                    row_thresholds.append(dyn_thresh)
                    # Pass if Value > Dynamic Threshold
                    passed = val >= dyn_thresh
                else:
                    passed = comparison_func(val, threshold)

                bar_colors.append("#9EDBC5" if passed else '#FF4646')

            ax = axes_flat[i]
            # Convert to numeric first (coercing errors to NaN), then fill
            clean_column_data = pd.to_numeric(data[col_name], errors='coerce').fillna(0)
            rects = ax.bar(data.index, clean_column_data, color=bar_colors)
            ax.axhline(y=0, color="#000000", linestyle='-', linewidth=0.5) 

            # --- Drawing Threshold Lines ---
            label_text = ""
            if is_dynamic_eps:
                label_text = "Target: 10% of CAGR"
                # Draw individual lines per bar
                x_starts = [r.get_x() for r in rects]
                x_ends = [r.get_x() + r.get_width() for r in rects]
                ax.hlines(y=row_thresholds, xmin=x_starts, xmax=x_ends, color="blue", linestyle='--')
                # Add text labels for dynamic values
                for x, th in zip(x_ends, row_thresholds):
                    ax.text(x, th, f' {th:.2f}', color='blue', fontsize=7, va='center')
                    
            elif op_str == "between":
                low, high = threshold
                ax.axhspan(low, high, color='blue', alpha=0.1)
                ax.axhline(low, color="blue", linestyle='--', linewidth=0.8)
                ax.axhline(high, color="blue", linestyle='--', linewidth=0.8)
                label_text = f"Target: {low} - {high}"
            else:
                ax.axhline(y=threshold, color="blue", linestyle='--', linewidth=1)
                label_text = f"Target: {op_str} {threshold}"

            # --- Value Labels on Bars ---
            for rect in rects:
                height = rect.get_height()
                # Only show label if height is not 0 (or wasn't originally None)
                display_label = f'{height:.2f}' if height != 0 else "N/A"
                ax.text(rect.get_x() + rect.get_width()/2., height, display_label, 
                        ha='center', va='bottom', fontsize=8, fontweight='bold')

            # --- ENHANCED TITLE with Units & Comments ---
            # Title: Metric Name
            # Subtitle (in smaller font): (Unit) Description
            ax.set_title(col_name, fontweight='bold', fontsize=11)
            
            # Add subtitle manually using text coordinates relative to axes
            subtitle = f"({unit}) {desc}"
            # Truncate if too long for clean display
            if len(subtitle) > 40: subtitle = subtitle[:37] + "..."
                
            ax.text(0.5, 0.98, subtitle, transform=ax.transAxes, 
                    ha='center', va='top', fontsize=8, color='gray', style='italic')

            ax.tick_params(axis='x', rotation=45, labelsize=8)
            
            # Legend
            legend_elements = [
                Patch(facecolor="#9EDBC5", label='Pass'),
                Patch(facecolor='#FF4646', label='Fail'),
                Patch(facecolor='blue', alpha=0.3, label=label_text)
            ]
            ax.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(1,1), fontsize=7)

        # Clear empty plots
        for k in range(i + 1, len(axes_flat)): axes_flat[k].axis('off')

        plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"An error occurred in plotting: {e}")
        import traceback
        traceback.print_exc()



## Sector and Industry Pie chart

In [34]:
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf

def print_sector_industry(fa_df: pd.DataFrame):
    sector_counts = fa_df['Sector'].value_counts()
    industry_counts = fa_df['Industry'].value_counts()

    labels_sector = sector_counts.index
    labels_industry = industry_counts.index

    sizes_sector = sector_counts.values
    sizes_industry = industry_counts.values

    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15,7))
    axs = axs.flatten()
    axs[0].pie(sizes_sector, labels=labels_sector, autopct='%1.1f%%', startangle=0, shadow=False)
    axs[0].set_title(f"Investements by Sector")
    axs[1].pie(sizes_industry, labels=labels_industry, autopct='%1.1f%%', startangle=0, shadow=False)
    axs[1].set_title(f"Investements by Industry")
    plt.axis('equal')
    plt.tight_layout()
    plt.show()

    return

## Scatter plot: Scores vs Beta

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import operator

def plot_risk_reward(fa_df, ratio_criteria):
    """
    Plots Score % (Reward) against Beta (Risk).
    Labels: High Weight (with star), Growth Satellite, and Watchlist.
    Includes the Strategy Guide Box.
    """
    plot_data = []
    ops = {">": operator.gt, "<": operator.lt, ">=": operator.ge, "<=": operator.le}
    
    for ticker in fa_df.index:
        total_passed = 0
        valid_metrics = [m for m in ratio_criteria.keys() if m in fa_df.columns]
        
        for metric in valid_metrics:
            val = fa_df.loc[ticker, metric]
            meta = ratio_criteria[metric]
            op_str, threshold = meta['criteria']
            
            # --- SAFETY CHECK 1: Missing Metric Data ---
            if pd.isna(val):
                passed = False
            elif op_str == "between":
                passed = threshold[0] <= val <= threshold[1]
            else:
                if 'EPS_usd' in metric and threshold == 10.0:
                    cagr = fa_df.loc[ticker, 'CAGR_%']
                    # Safety check for CAGR too
                    target = (cagr * 0.10) if not pd.isna(cagr) else 0
                    # target = fa_df.loc[ticker, 'CAGR_%'] * 0.10 ***************************************************
                    passed = val >= target
                else:
                    passed = ops[op_str](val, threshold)
            
            if passed: total_passed += 1
            
        score_pct = (total_passed / len(valid_metrics)) * 100 if valid_metrics else 0

        plot_data.append({
            'Ticker': ticker,
            'Score %': score_pct,
            # # --- SAFETY CHECK 2: Missing Beta ---
            'Beta': fa_df.loc[ticker, 'Beta'] if 'Beta' in fa_df.columns else 1.0,
            'Sector': fa_df.loc[ticker, 'Sector'] if 'Sector' in fa_df.columns else 'N/A'
        })

    # --- SAFETY CHECK 3: Handle empty DataFrames ---
    if not plot_data:
        print("⚠️ No data available to plot Risk/Reward.")
        return

    scored_df = pd.DataFrame(plot_data).dropna(subset=['Beta', 'Score %'])
    avg_score = scored_df['Score %'].mean()

    plt.figure(figsize=(14, 9))
    
    # 1. Base Scatter Plot by Sector
    sectors = sorted(scored_df['Sector'].unique())
    cmap = plt.get_cmap('tab20')
    colors = [cmap(i) for i in np.linspace(0, 1, len(sectors))]
    sector_colors = dict(zip(sectors, colors))
    
    for sector in sectors:
        subset = scored_df[scored_df['Sector'] == sector]
        plt.scatter(
            subset['Beta'], subset['Score %'], 
            s=160, label=sector, color=sector_colors[sector], 
            alpha=0.6, edgecolors='black', linewidth=0.5
        )

    # 2. Strategy Quadrant Lines
    plt.axvline(x=1.0, color='red', linestyle='--', alpha=0.3, label='Market Risk (Beta=1.0)')
    plt.axhline(y=avg_score, color='blue', linestyle='--', alpha=0.3, label=f'Avg Score ({avg_score:.1f}%)')

    # 3. Annotations with Sub-Labels
    for i, row in scored_df.iterrows():
        # Main Ticker Name
        plt.annotate(
            row['Ticker'], (row['Beta'], row['Score %']), 
            xytext=(0, 12), textcoords='offset points', 
            ha='center', fontsize=10, fontweight='bold'
        )
        
        # CATEGORIZATION LOGIC
        if row['Score %'] > avg_score:
            if row['Beta'] < 1.0:
                # Putting the star marker symbol next to the text
                label_text = "* High Weight" 
                color = 'darkgreen'
            else:
                label_text = "Growth Satellite"
                color = 'darkorange'
        else:
            label_text = "Watchlist"
            color = 'red'
            
        plt.annotate(
            label_text, (row['Beta'], row['Score %']), 
            xytext=(0, -18), textcoords='offset points', 
            ha='center', fontsize=8, color=color, fontweight='bold', alpha=0.8
        )

    # 4. Strategy Guide Box (Recommendation Box)
    x_min, x_max = plt.xlim()
    y_min, y_max = plt.ylim()
    
    props = dict(boxstyle='round', facecolor='white', alpha=0.8, edgecolor='gray')
    strategy_text = (
        "STRATEGY GUIDE\n"
        "* High Weight: Fundamentally strong & Low volatility\n"
        "Growth Satellite: Strong fundamentals but high market risk\n"
        "Watchlist: Currently failing one or more core criteria"
    )
    plt.text(x_min + (x_max-x_min)*0.02, y_min + (y_max-y_min)*0.02, 
             strategy_text, fontsize=9, verticalalignment='bottom', bbox=props)

    # 5. Styling & Quadrant Labels
    plt.text(x_min + (x_max-x_min)*0.02, y_max * 0.96, "SAFE HAVENS\nLow Beta, High Score", color='green', fontsize=10, fontweight='bold', va='top')
    plt.text(x_max - (x_max-x_min)*0.02, y_max * 0.96, "AGGRESSIVE GROWTH\nHigh Beta, High Score", color='orange', fontsize=10, fontweight='bold', va='top', ha='right')

    plt.title('Investment Strategy: Risk (Beta) vs. Reward (Score %)', fontsize=18, fontweight='bold', pad=25)
    plt.xlabel('Risk (Beta relative to S&P 500)', fontsize=13)
    plt.ylabel('Reward (Fundamental Score %)', fontsize=13)
    plt.grid(True, linestyle=':', alpha=0.4)
    plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', title="Sectors", frameon=True)
    
    plt.tight_layout()
    plt.show()

## Top N Assets and Plot Sector Treemap

In [ ]:
import pandas as pd
import operator
import plotly.express as px

def get_top_N_assets(fa_df, ratio_criteria, top_n=10):
    """
    Calculates scores in 'long' format and returns the Top N assets.
    Compatible with sorting, grouping, and Treemaps.
    """
    tickers = fa_df.index
    scored_list = []
    ops = {">": operator.gt, "<": operator.lt, ">=": operator.ge, "<=": operator.le}
    
    for ticker in tickers:
        total_passed = 0
        valid_metrics = [m for m in ratio_criteria.keys() if m in fa_df.columns]
        
        for metric in valid_metrics:
            val = fa_df.loc[ticker, metric]
            meta = ratio_criteria[metric]
            op_str, threshold = meta['criteria']
            
            # --- THE SAFETY CHECK ---
            if pd.isna(val):
                passed = False # Fail if data is missing
            elif op_str == "between":
                passed = threshold[0] <= val <= threshold[1]
            else:
                if 'EPS_usd' in metric and threshold == 10.0:
                    cagr = fa_df.loc[ticker, 'CAGR_%']
                    # Check if CAGR is also missing
                    cagr_val = cagr if not pd.isna(cagr) else 0
                    target = cagr_val * 0.10
                    passed = val >= target
                else:
                    passed = ops[op_str](val, threshold)
            
            if passed: total_passed += 1
            
        score_pct = (total_passed / len(valid_metrics)) * 100 if valid_metrics else 0
        
        # Build record with safety for missing Name/Sector info
        scored_list.append({
            'Ticker': ticker,
            'Name': fa_df.loc[ticker, 'Name'] if 'Name' in fa_df.columns else ticker,
            'Sector': fa_df.loc[ticker, 'Sector'] if 'Sector' in fa_df.columns and not pd.isna(fa_df.loc[ticker, 'Sector']) else 'N/A',
            'Industry': fa_df.loc[ticker, 'Industry'] if 'Industry' in fa_df.columns and not pd.isna(fa_df.loc[ticker, 'Industry']) else 'N/A',
            'Score %': score_pct
        })

    # Create the DataFrame and get Top N
    scored_df = pd.DataFrame(scored_list)
    
    # Optional: if you have no data, return an empty DF to prevent Treemap from crashing
    if scored_df.empty:
        return pd.DataFrame(columns=['Ticker', 'Name', 'Sector', 'Industry', 'Score %'])

    top_assets = scored_df.nlargest(top_n, 'Score %')
    
    return top_assets



def plot_sector_treemap(top_assets):
    """
    Creates a Treemap grouping Top Assets by Sector and Industry
    """
    fig = px.treemap(
        top_assets, 
        path=[px.Constant("Top Performers"), 'Sector', 'Industry', 'Ticker'], 
        values='Score %',
        color='Score %',
        color_continuous_scale='Greens', # Red-Yellow-Green 'RdYlGn'
        hover_data=['Name', 'Score %'],
        title=f"Top {len(top_assets)} Assets overall"
    )
    
    fig.update_traces(textinfo="label+value")
    fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))
    fig.show()


## Top N per Sector and Industry

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import operator

def top_N_sector_industry(fa_df, ratio_criteria, n_per_sector=10):
    """
    Generates side-by-side Table + Chart for each sector.
    Calculates scores in 'long format' to avoid transposition errors.
    """
    # 1. GENERATE SCORES IN LONG FORMAT (Rows = Tickers)
    tickers = fa_df.index
    scored_list = []
    ops = {">": operator.gt, "<": operator.lt, ">=": operator.ge, "<=": operator.le}
    
    for ticker in tickers:
        total_passed = 0
        valid_metrics = [m for m in ratio_criteria.keys() if m in fa_df.columns]
        
        for metric in valid_metrics:
            val = fa_df.loc[ticker, metric]
            meta = ratio_criteria[metric]
            op_str, threshold = meta['criteria']
            
            # --- THE SAFETY CHECK ---
            # If data is missing, it doesn't pass
            if pd.isna(val):
                passed = False  
            elif op_str == "between":
                passed = threshold[0] <= val <= threshold[1]
            else:
                if 'EPS_usd' in metric and threshold == 10.0:
                    cagr = fa_df.loc[ticker, 'CAGR_%']
                    # Safety for missing CAGR
                    cagr_val = cagr if not pd.isna(cagr) else 0
                    target = cagr_val * 0.10
                    passed = val >= target
                else:
                    passed = ops[op_str](val, threshold)
            
            if passed: total_passed += 1
            
        score_pct = (total_passed / len(valid_metrics)) * 100 if valid_metrics else 0

        scored_list.append({
            'Ticker': ticker,
            'Sector': fa_df.loc[ticker, 'Sector'] if 'Sector' in fa_df.columns and not pd.isna(fa_df.loc[ticker, 'Sector']) else 'Unknown',
            'Industry': fa_df.loc[ticker, 'Industry'] if 'Industry' in fa_df.columns and not pd.isna(fa_df.loc[ticker, 'Industry']) else 'Unknown',
            'Score %': score_pct
        })

    # Create the DataFrame and sort by score
    scored_df = pd.DataFrame(scored_list).sort_values('Score %', ascending=False)
    
    # 2. Get unique sectors to iterate through
    sectors = sorted(scored_df['Sector'].unique())

    for sector in sectors:
        # Filter for the current sector
        sector_subset = scored_df[scored_df['Sector'] == sector]
        sector_avg = sector_subset['Score %'].mean()
        
        # Get Top N for this sector
        top_n = sector_subset.head(n_per_sector)

        # Create Figure
        fig, (ax_table, ax_chart) = plt.subplots(1, 2, figsize=(20, 6), 
                                                gridspec_kw={'width_ratios': [1, 1.2]})
        
        # --- LEFT SIDE: TABLE ---
        ax_table.axis('off')
        table_cols = ['Ticker', 'Industry', 'Score %']
        # Format the score to include the % sign for the table display
        display_subset = top_n.copy()
        display_subset['Score %'] = display_subset['Score %'].apply(lambda x: f"{x:.1f}%")
        table_vals = display_subset[table_cols].values
        
        table = ax_table.table(cellText=table_vals, colLabels=table_cols, 
                               loc='center', cellLoc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(11)
        table.scale(1, 2.2) 
        ax_table.set_title(f"TOP {n_per_sector} PERFORMERS: {sector.upper()}", 
                           fontweight='bold', fontsize=16, pad=30)

        # --- RIGHT SIDE: CHART ---
        # Color based on score (Red to Green)
        norm = plt.Normalize(0, 100)
        colors = plt.cm.RdYlGn(norm(top_n['Score %'])) 
        
        bars = ax_chart.barh(top_n['Ticker'], top_n['Score %'], color=colors, 
                             edgecolor='black', alpha=0.8)
        ax_chart.invert_yaxis() 
        
        # Sector Average Vertical Line
        ax_chart.axvline(sector_avg, color='blue', linestyle='--', linewidth=2, 
                         label=f'Sector Avg: {sector_avg:.1f}%')
        
        # Formatting
        ax_chart.set_xlim(0, 110)
        ax_chart.set_xlabel("Financial Ratios Score (%)", fontsize=12)
        ax_chart.set_title(f"Scoring Analysis - {sector}", fontsize=14)
        ax_chart.legend(loc='lower right')
        
        # Add labels to bars
        for bar in bars:
            ax_chart.text(bar.get_width() + 2, bar.get_y() + bar.get_height()/2, 
                          f'{bar.get_width():.1f}%', va='center', fontweight='bold')

        plt.tight_layout()
        plt.show()

## Best in-class by SECTOR

In [ ]:
import pandas as pd
import operator

def top_1_sector(fa_df, ratio_criteria):
    """
    Picks the top 1 asset per Sector based on the final Score %.
    Calculates scores in 'long format' to avoid transposition errors.
    """
    # 1. CALCULATE SCORES IN LONG FORMAT
    tickers = fa_df.index
    scored_list = []
    ops = {">": operator.gt, "<": operator.lt, ">=": operator.ge, "<=": operator.le}
    
    for ticker in tickers:
        total_passed = 0
        valid_metrics = [m for m in ratio_criteria.keys() if m in fa_df.columns]
        
        for metric in valid_metrics:
            val = fa_df.loc[ticker, metric]
            meta = ratio_criteria[metric]
            op_str, threshold = meta['criteria']
            
            # --- SAFETY CHECK: Missing Metric Data ---
            if pd.isna(val):
                passed = False
            elif op_str == "between":
                passed = threshold[0] <= val <= threshold[1]
            else:
                if 'EPS_usd' in metric and threshold == 10.0:
                    target = fa_df.loc[ticker, 'CAGR_%'] * 0.10
                    passed = val >= target
                else:
                    passed = ops[op_str](val, threshold)
            
            if passed: total_passed += 1
            
        score_pct = (total_passed / len(valid_metrics)) * 100 if valid_metrics else 0

        scored_list.append({
            'Ticker': ticker,
            'Sector': fa_df.loc[ticker, 'Sector'] if 'Sector' in fa_df.columns and not pd.isna(fa_df.loc[ticker, 'Sector']) else 'Unknown',
            'Industry': fa_df.loc[ticker, 'Industry'] if 'Industry' in fa_df.columns and not pd.isna(fa_df.loc[ticker, 'Industry']) else 'Unknown',
            'Score %': score_pct
        })

    # 2. Create DataFrame and Sort
    scored_df = pd.DataFrame(scored_list).sort_values('Score %', ascending=False)
    
    # 3. Pick Top 1 per Sector
    # .groupby('Sector').head(1) works because we just sorted by Score %
    best_in_class = scored_df.groupby('Sector').head(1)
    
    return best_in_class[['Sector', 'Industry', 'Ticker', 'Score %']]


## Portfolio Summary - Beta and Scores

In [ ]:
import pandas as pd
import operator

def display_combined_strategy_comparison(fa_df, ratio_criteria, top_n=10):
    """
    Calculates and displays a side-by-side comparison of three portfolio strategies
    including a list of tickers for each.
    """
    
    # --- Internal helper to calculate stats for a specific subset ---
    def get_stats(subset_df):
        if subset_df.empty:
            return {
                "Total Tickers": 0,
                "Avg Score_%": 0,
                "Avg Beta": 0,
                "Character": "N/A",
                "Included Tickers": ""
            }
        avg_score = subset_df['Score %'].mean()
        avg_beta = subset_df['Beta'].dropna().mean()
        ticker_list = ", ".join(subset_df['Ticker'].tolist())
        
        if pd.isna(avg_beta): character = "UNKNOWN"
        if avg_beta < 0.8: character = "CONSERVATIVE / DEFENSIVE"
        elif avg_beta < 1.1: character = "MARKET-NEUTRAL"
        else: character = "AGGRESSIVE / HIGH VOLATILITY"
            
        return {
            "Total Tickers": int(len(subset_df)),
            "Avg Score_%": avg_score,
            "Avg Beta": avg_beta,
            "Character": character,
            "Included Tickers": ticker_list
        }

    # 1. Generate the base 'Long Format' scored data
    tickers = fa_df.index
    scored_list = []
    ops = {">": operator.gt, "<": operator.lt, ">=": operator.ge, "<=": operator.le}
    
    for ticker in tickers:
        total_passed = 0
        valid_metrics = [m for m in ratio_criteria.keys() if m in fa_df.columns]
        for metric in valid_metrics:
            val = fa_df.loc[ticker, metric]
            meta = ratio_criteria[metric]
            op_str, threshold = meta['criteria']

            # --- SAFETY CHECK: Handle missing data (None/NaN) ---
            if pd.isna(val):
                passed = False
            elif op_str == "between": 
                passed = threshold[0] <= val <= threshold[1]
            else:
                if 'EPS_usd' in metric and threshold == 10.0:
                    cagr = fa_df.loc[ticker, 'CAGR_%']
                    # Safety check for CAGR too
                    target = (cagr * 0.10) if not pd.isna(cagr) else 0
                    passed = val >= target
                else: passed = ops[op_str](val, threshold)
            if passed: total_passed += 1
            
        scored_list.append({
            'Ticker': ticker,
            'Sector': fa_df.loc[ticker, 'Sector'] if 'Sector' in fa_df.columns else 'N/A',
            'Score %': (total_passed / len(valid_metrics)) * 100 if valid_metrics else 0,
            'Beta': pd.to_numeric(fa_df.loc[ticker, 'Beta'], errors='coerce') if 'Beta' in fa_df.columns else 1.0
        })

    full_df = pd.DataFrame(scored_list).sort_values('Score %', ascending=False)

    # 2. Define the three strategies
    strategies = {
        "All Assets": full_df,
        f"Top {top_n} Overall": full_df.head(top_n),
        "Best-in-Class (Sector)": full_df.groupby('Sector').head(1)
    }

    # 3. Build the comparison table
    comparison_data = {}
    for name, df in strategies.items():
        comparison_data[name] = get_stats(df)

    comparison_df = pd.DataFrame(comparison_data)

    # 4. Styling the table
    # Formatting numbers
    def format_vals(val):
        if isinstance(val, float):
            if val > 5: return f"{val:.1f}%" # Scores
            return f"{val:.2f}"             # Beta
        return val

    styled_comparison = comparison_df.map(format_vals).style.set_table_styles([
        {'selector': 'th.row_heading', 'props': [('text-align', 'left'), ('font-weight', 'bold')]},
        {'selector': 'th.col_heading', 'props': [('text-align', 'center'), ('background-color', '#0D6654'), ('color', 'white')]}
    ]).set_properties(**{'text-align': 'center', 'padding': '10px'}) \
      .set_properties(subset=pd.IndexSlice[['Avg Score_%', 'Avg Beta'], :], **{'font-weight': 'bold'}) \
      .set_properties(subset=pd.IndexSlice[['Included Tickers'], :], **{'font-size': '9pt', 'width': '250px'})

    print("--- STRATEGY COMPARISON MATRIX ---")
    return styled_comparison

## Dividends, Splits and Recommendations

In [ ]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import logging

# This silences the internal yfinance logger warnings in the console
logging.getLogger('yfinance').setLevel(logging.CRITICAL)

def print_dividends_splits_recommendations(ticker_list, start_date, today):
    """
    Final Clean Version:
    - Fixed Seaborn FutureWarnings
    - Handles Delisted Tickers gracefully
    - Integrated Dashboard Layout
    """
    all_divs = []
    all_splits = []
    summary_data = []

    print(f"Gathering intelligence for {len(ticker_list)} tickers...")

    for t_name in ticker_list:
        try:
            t = yf.Ticker(t_name)
            
            # Pre-flight check: see if the ticker actually exists/has recent data
            # This prevents the 404 errors from clogging the output
            hist = t.history(period="1d")
            if hist.empty:
                print(f"Skipping {t_name}: Ticker appears to be delisted or inactive.")
                continue

            # --- 1. Dividends ---
            divs = t.dividends
            annual_div_12m = 0.0
            if not divs.empty:
                divs.index = divs.index.tz_localize(None)
                one_year_ago = today - timedelta(days=365)
                annual_div_12m = divs.loc[divs.index >= one_year_ago].sum()
                
                mask_evol = (divs.index >= start_date) & (divs.index <= today)
                if any(mask_evol):
                    div_subset = divs.loc[mask_evol].reset_index()
                    div_subset['Ticker'] = t_name
                    all_divs.append(div_subset)
            
            # --- 2. Splits (ALL TIME) ---
            splits = t.splits
            if not splits.empty:
                splits.index = splits.index.tz_localize(None)
                split_subset = splits.reset_index()
                split_subset['Ticker'] = t_name
                all_splits.append(split_subset)
                
            # --- 3. Recommendations ---
            try:
                rec = t.recommendations
                row_info = {'Ticker': t_name, 'Annual Div (12m)': annual_div_12m}
                # Check if it's a valid DataFrame with rows
                if isinstance(rec, pd.DataFrame) and not rec.empty:
                    # Get the most recent row
                    latest = rec.tail(1).iloc[0].to_dict()
                    for key in ['strongBuy', 'buy', 'hold', 'sell', 'strongSell']:
                        row_info[key] = latest.get(key, 0)
                else:
                    raise ValueError("No recs")
            except:
                for key in ['strongBuy', 'buy', 'hold', 'sell', 'strongSell']:
                    row_info[key] = 0
            summary_data.append(row_info)

        except Exception:
            # Silently skip tickers that cause hard HTTP errors
            continue

    if not summary_data:
        print("No valid data found for the provided ticker list.")
        return

    # --- PLOTTING ---
    fig = plt.figure(figsize=(22, 28))
    grid = plt.GridSpec(3, 2, hspace=0.35, wspace=0.25)

    # AX 1: ANNUAL DIVIDEND BAR CHART (Row 0, Col 0)
    ax_bar = fig.add_subplot(grid[0, 0])
    df_sum = pd.DataFrame(summary_data).sort_values('Annual Div (12m)', ascending=False)
    
    # FIXED: Added hue='Ticker' and legend=False to resolve FutureWarning
    sns.barplot(data=df_sum, x='Ticker', y='Annual Div (12m)', ax=ax_bar, 
                palette="Greens_r", edgecolor='black', hue='Ticker', legend=False)
    
    ax_bar.set_title("Annual Dividend Amount (Last 12 Months)", fontsize=16, fontweight='bold')
    ax_bar.set_ylabel("USD Total per Share")
    ax_bar.tick_params(axis='x', rotation=45)
    
    # AX 2: ANALYST HEATMAP (Row 0, Col 1)
    ax_heat = fig.add_subplot(grid[0, 1])
    # Create DF and ensure all recommendation columns exist
    cols = ['strongBuy', 'buy', 'hold', 'sell', 'strongSell']
    df_rec_plot = pd.DataFrame(summary_data).set_index('Ticker')
    df_rec_plot = df_rec_plot[[c for c in cols if c in df_rec_plot.columns]]

    # Fill missing columns with 0 if a ticker has no recs
    for c in cols:
        if c not in df_rec_plot.columns:
            df_rec_plot[c] = 0

    # Reorder columns for the heatmap
    df_rec_plot = df_rec_plot[cols]

    # NORMALIZATION: Scale by row (ticker) so we see the relative preference per stock
    # This prevents the "Inconsistent Shape" error
    row_sums = df_rec_plot.sum(axis=1)
    df_norm = df_rec_plot.div(row_sums, axis=0).fillna(0)
    
    sns.heatmap(df_norm, annot=df_rec_plot, fmt='g', cmap="YlGn", ax=ax_heat, cbar=False)
    ax_heat.set_title("Analysts Recommendations (Scaled by Ticket)", fontsize=16, fontweight='bold')

    # AX 3: DIVIDEND EVOLUTION (Row 1, Full Width)
    ax_evol = fig.add_subplot(grid[1, :])
    if all_divs:
        df_div_long = pd.concat(all_divs)
        sns.lineplot(data=df_div_long, x='Date', y='Dividends', hue='Ticker', 
                     marker='o', markersize=8, ax=ax_evol, alpha=0.7)
        ax_evol.set_title(f"Dividend Payout Evolution (Since {start_date.date()})", fontsize=16, fontweight='bold')
        ax_evol.set_ylabel("USD Single Payment")
        ax_evol.legend(bbox_to_anchor=(1.01, 1), loc='upper left', ncol=2, fontsize='small')
        ax_evol.grid(True, alpha=0.2)

    # AX 4: ALL-TIME SPLIT BUBBLES (Row 2, Full Width)
    ax_split = fig.add_subplot(grid[2, :])
    # Filter tickers to only those that survived the pre-flight check
    valid_tickers = sorted(df_sum['Ticker'].tolist())
    ax_split.set_yticks(range(len(valid_tickers)))
    ax_split.set_yticklabels(valid_tickers)
    
    if all_splits:
        df_split_long = pd.concat(all_splits)
        # Only plot splits for tickers still in our valid list
        df_split_long = df_split_long[df_split_long['Ticker'].isin(valid_tickers)]
        
        sns.scatterplot(data=df_split_long, x='Date', y='Ticker', size='Stock Splits', 
                        hue='Ticker', sizes=(100, 2000), ax=ax_split, legend=False, alpha=0.5)
        for i in range(len(df_split_long)):
            ax_split.text(df_split_long.Date.iloc[i], df_split_long.Ticker.iloc[i], 
                          f"{df_split_long['Stock Splits'].iloc[i]}:1", 
                          fontweight='bold', va='center', ha='center', fontsize=8)
    
    ax_split.set_title("All-Time Historical Stock Splits (Size = Ratio)", fontsize=16, fontweight='bold')
    ax_split.grid(True, axis='x', alpha=0.1)

    plt.tight_layout()
    plt.show()

## Plot Prices

In [41]:
import plotly.express as px
import pandas as pd
import numpy as np

def plot_prices(prices:pd.DataFrame, yaxis_label:str):

    # 1. Identify numeric columns
    # Ensure we only plot columns containing price data
    data_cols = prices.select_dtypes(include=[np.number]).columns.tolist()
    no_assets = len(data_cols)

    # 2. Prepare the Line Data for Plotly (Long Format)
    df_melted = prices[data_cols].reset_index().melt(
        id_vars=prices.index.name or 'index', 
        var_name='Asset', 
        value_name='Price'
    )

    # Standardize the date column name for the x-axis
    date_col = df_melted.columns[0]
    df_melted.rename(columns={date_col: 'Date'}, inplace=True)

    # 3. Create the Interactive Plot
    fig = px.line(
        df_melted, 
        x='Date', 
        y='Price', 
        color='Asset',
        title=f"Daily Asset Prices ({no_assets} Assets)",
        height=1000, 
        template='plotly_white'
    )

    # 4. Configure Individual Hover Labels (White background, USD format)
    fig.update_traces(
        hoverinfo='all', 
        hovertemplate="<b>%{fullData.name}</b><br>Price: $%{y:,.2f}$<extra></extra>",
        selector=dict(type='scatter')
    )

    # 5. Layout Customization
    fig.update_layout(
        hovermode='x', # Triggers labels and the vertical line based on cursor x-position
        yaxis_title=yaxis_label,#"Price (USD)",
        xaxis_title="Date",
        
        # --- WHITE HOVER BOX CUSTOMIZATION ---
        hoverlabel=dict(
            bgcolor="white",
            font_size=12,
            font_family="Arial",
            font_color="black",
            bordercolor="#dddddd"
        ),
        
        # --- X-AXIS: RED CROSSHAIR + MONTHLY TICKS ---
        xaxis=dict(
            showspikes=True,
            spikemode='across',
            spikethickness=3,
            spikecolor="red",
            spikedash="dot",
            rangeslider=dict(visible=True), # Zoom slider at bottom
            type="date",
            
            # Monthly interval settings
            dtick="M1",             # Force a tick mark for every 1 month
            tickformat="%b %d, %Y",     # Display as 'Jan 02, 2024'
            tickangle=-90,          # Tilt labels to prevent overlap
            nticks=24,              # Hint to increase total number of labels
            gridcolor="#eeeeee"     # Light vertical grid lines for readability
        ),
        
        # --- Y-AXIS FORMATTING ---
        yaxis=dict(
            gridcolor="#eeeeee",
            tickformat=".2f"        # Standard decimal pricing
        ),
        
        # --- LEGEND SETTINGS ---
        legend=dict(
            orientation="v",
            x=1.02,
            font=dict(size=9),
            itemclick="toggle",              # Click once to hide/show
            itemdoubleclick="toggleothers"   # Double-click to isolate
        ),
        
        # --- INTERACTIVE BUTTONS ---
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                x=0.5, y=1.1, xanchor="center",
                buttons=list([
                    dict(
                        label="Show All Assets", 
                        method="restyle", 
                        args=[{"visible": [True] * no_assets}]
                    ),
                    dict(
                        label="Clear All (Selection Mode)", 
                        method="restyle", 
                        args=[{"visible": ["legendonly"] * no_assets}]
                    )
                ]),
            )
        ],
        margin=dict(r=150, l=50, b=100, t=120)
    )

    # 6. Show the plot
    fig.show()

    return


## Plot Returns Distributions

In [42]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

def plot_returns_distributions(df: pd.DataFrame, num_cols:int=4):
    """
    1. Plots a grid of distribution plots with advanced moments.
    2. Includes instructional footer with fixed padding to avoid overlap.
    3. Displays a statistical ranking table.
    """
    columns = df.columns
    num_assets = len(columns)
    
    # --- Part 1: Plotting the Matrix ---
    num_rows = int(np.ceil(num_assets / num_cols))
    
    # Increase the +2 to +3 in figsize to give the footer more breathing room
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(num_cols * 4, num_rows * 3.5 + 3))
    axes_flat = axes.flatten()
    
    stats_data = []

    for i, ticker in enumerate(columns):
        ax = axes_flat[i]
        data = df[ticker].dropna()
        
        # Plot
        sns.histplot(data, kde=True, ax=ax, color='teal', bins=30, alpha=0.4)
        
        # Calculate Stats
        mean_val = data.mean()
        median_val = data.median()
        std_val = data.std()
        skew_val = data.skew()
        kurt_val = data.kurtosis() 
        
        stats_data.append({
            'Ticker': ticker, 'Mean': mean_val, 'Median': median_val,
            'Std Dev': std_val, 'Skewness': skew_val, 'Kurtosis': kurt_val
        })
        
        # Vertical Lines
        ax.axvline(mean_val, color='red', linestyle='--', linewidth=1.5)
        ax.axvline(median_val, color='gold', linestyle='-', linewidth=1.5)
        ax.axvline(0, color='black', linestyle=':', linewidth=1)
        
        # Styling
        ax.set_title(f"{ticker}", fontsize=14, fontweight='bold')
        ax.set_xlabel("Daily Return")
        ax.set_ylabel("Freq")
        
        stats_text = (
            f"Mean:   {mean_val:.3f}\n"
            f"Median: {median_val:.3f}\n"
            f"Std:    {std_val:.3f}\n"
            f"Skew:   {skew_val:.2f}\n"
            f"Kurt:   {kurt_val:.2f}"
        )
        
        ax.text(0.97, 0.95, stats_text, transform=ax.transAxes, 
                verticalalignment='top', horizontalalignment='right',
                bbox=dict(boxstyle='round,pad=0.5', facecolor='white', alpha=0.8, edgecolor='#cccccc'), 
                fontsize=8, family='monospace')

    # Cleanup unused axes
    for j in range(i + 1, len(axes_flat)):
        axes_flat[j].axis('off')
    
    # Instructional Footer
    notes = (
        "INTERPRETATION GUIDE:\n"
        "• SKEWNESS: Positive (>0) = Tail on the right (occasional big gains). "
        "Negative (<0) = Tail on the left (occasional big crashes).\n"
        "• KURTOSIS: High (Excess >0) = 'Fat Tails'. Higher probability of extreme price movements (outliers) than a normal distribution.\n"
        "• MEAN vs MEDIAN: If Mean (Red) > Median (Gold), the distribution is positively skewed, pulled by large positive outliers.\n"
        "• ZERO LINE: The dotted black line represents 0% return."
    )
    
    # Use figtext but ensure the bottom of the subplots is pushed up
    plt.figtext(0.01, 0.02, notes, fontsize=11, family='sans-serif', 
                bbox=dict(facecolor='#f9f9f9', alpha=0.9, edgecolor='#dddddd', boxstyle='round,pad=1'),
                verticalalignment='bottom', horizontalalignment='left')

    plt.suptitle('Portfolio Daily Returns: Distribution & Statistical Moments', fontsize=22, y=0.98)
    
    # ADJUST SUBPLOTS: bottom=0.15 creates a 15% margin at the bottom for the notes
    plt.subplots_adjust(bottom=0.12, top=0.92, hspace=0.4, wspace=0.3)
    
    plt.show()

    # --- Part 2: Ranking Table ---
    stats_df = pd.DataFrame(stats_data).set_index('Ticker')
    print("\n" + "="*80)
    print("FAT-TAIL RISK RANKING (Sorted by Highest Kurtosis)")
    print("="*80)
    
    styled_df = stats_df.sort_values('Kurtosis', ascending=False).style.format(precision=4).background_gradient(
        subset=['Kurtosis'], cmap='YlOrRd'
    )
    
    display(styled_df)
    return stats_df

## Plot Daily Returns

In [43]:
import plotly.express as px
import pandas as pd
import numpy as np

def plot_daily_returns(daily_returns:pd.DataFrame):
    # Round and prepare data
    daily_returns_perc = round(daily_returns * 100, 1)
    data_cols = daily_returns_perc.select_dtypes(include=[np.number]).columns.tolist()

    # 2. Pre-calculate the Top 10 Hover Text
    def get_top_10_labels(row):
        top_10 = row[data_cols].sort_values(ascending=False).head(10)
        return "<br>".join([f"{i+1}. {ticker}: {val:.1f}%" for i, (ticker, val) in enumerate(top_10.items())])

    # Internal helper columns
    daily_returns_perc['Top_10_Hover'] = daily_returns_perc.apply(get_top_10_labels, axis=1)
    hover_anchor_y = daily_returns_perc[data_cols].max().max() * 1.1 

    # 3. Prepare the Line Data
    df_melted = daily_returns_perc[data_cols].reset_index().melt(
        id_vars=daily_returns_perc.index.name or 'index', 
        var_name='Asset', 
        value_name='Daily Return'
    )
    df_melted.rename(columns={df_melted.columns[0]: 'Date'}, inplace=True)

    # 4. Create the Interactive Plot
    no_assets = len(data_cols)
    fig = px.line(
        df_melted, 
        x='Date', 
        y='Daily Return', 
        color='Asset',
        title=f"Daily Returns Analysis ({no_assets} Assets) - Top 10 Ranking",
        height=1000, 
        template='plotly_white'
    )

    # 5. Configure Specific Asset Hover
    fig.update_traces(
        hoverinfo='all', 
        hovertemplate="<b>%{fullData.name}</b><br>Return: %{y:.1f}%<extra></extra>",
        selector=dict(type='scatter')
    )

    # 6. Add the "Top 10" Helper Trace
    fig.add_scatter(
        x=daily_returns_perc.index,
        y=[hover_anchor_y] * len(daily_returns_perc),
        mode='markers',
        marker=dict(opacity=0), 
        name="Top 10 List",    
        customdata=daily_returns_perc['Top_10_Hover'],
        hovertemplate="<b>DAILY TOP 10 RANKING</b><br>%{customdata}<extra></extra>",
        showlegend=False        
    )

    # 7. Layout & Crosshair Customization
    fig.update_layout(
        hovermode='x', 
        yaxis_title="Daily Returns (%)",
        xaxis_title="Date",
        
        hoverlabel=dict(
            bgcolor="white",
            font_size=12,
            font_family="Arial",
            font_color="black",
            bordercolor="#dddddd"
        ),
        
        yaxis=dict(
            tickformat='.1f',  
            ticksuffix='%',    
            title="Daily Returns (%)",
            zeroline=True,
            zerolinewidth=3,
            zerolinecolor="#000000"
        ),

        # --- UPDATED XAXIS FOR MORE MONTHS ---
        xaxis=dict(
            showspikes=True,
            spikemode='across',
            spikethickness=3,
            spikecolor="red",
            spikedash="dot",
            rangeslider=dict(visible=True),
            type="date",
            dtick="M1",              # Forces a tick every 1 month
            tickformat="%b %d, %Y",  # Formats as 'Jan 02, 2024'
            tickangle=-90,           # Slanted for better readability
            nticks=24                # Hint to allow more labels if needed
        ),
        
        legend=dict(
            orientation="v",
            x=1.02,
            font=dict(size=9),
            itemclick="toggle",
            itemdoubleclick="toggleothers"
        ),
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                x=0.5, y=1.05, xanchor="center",
                buttons=list([
                    dict(label="Show All Assets", method="restyle", args=[{"visible": [True] * (no_assets + 1)}]),
                    dict(label="Clear All (Selection Mode)", method="restyle", args=[{"visible": ["legendonly"] * no_assets + [True]}])
                ]),
            )
        ],
        margin=dict(r=150, l=50, b=100, t=120)
    )

    fig.show()
    return


## Plot Cumulative Returns

In [44]:
import plotly.express as px
import pandas as pd
import numpy as np

def plot_cumulative_returns(cumulative_returns_input:pd.DataFrame):

    # 1. Work on a copy to avoid modifying the original variable outside this function
    df_local = cumulative_returns_input.copy()

    # 1. Prepare Data & Calculate Top 10 for each day
    data_cols = df_local.select_dtypes(include=[np.number]).columns.tolist()

    def get_top_10_labels(row):
        top_10 = row[data_cols].sort_values(ascending=False).head(10)
        return "<br>".join([f"{i+1}. {ticker}: {val:.1f}%" for i, (ticker, val) in enumerate(top_10.items())])

    # Internal helper data
    df_local['Top_10_Hover'] = df_local.apply(get_top_10_labels, axis=1)
    hover_anchor_y = df_local[data_cols].max().max() * 1.1 

    # Melt for line plotting
    df_melted = df_local[data_cols].reset_index().melt(
        id_vars=df_local.index.name or 'index', 
        var_name='Asset', 
        value_name='Cumulative Return'
    )
    df_melted.rename(columns={df_melted.columns[0]: 'Date'}, inplace=True)

    # 2. Create the Base Plot
    no_assets = len(data_cols)
    fig = px.line(
        df_melted, 
        x='Date', 
        y='Cumulative Return', 
        color='Asset',
        title=f"Cumulative Returns in % ({no_assets} Assets)",
        height=900, 
        template='plotly_white'
    )

    # 3. Configure the Specific Stock Hover
    fig.update_traces(
        hoverinfo='all', 
        hovertemplate="<b>%{fullData.name}</b><br>Return: %{y:,.1f}%<extra></extra>",
        selector=dict(type='scatter')
    )

    # 4. Add the "Top 10 Ranking" Helper Trace
    fig.add_scatter(
        x=df_local.index,
        y=[hover_anchor_y] * len(df_local),
        mode='markers',
        marker=dict(opacity=0), 
        name="Ranking Helper",    
        customdata=df_local['Top_10_Hover'],
        hovertemplate="<b>DAILY TOP 10</b><br>%{customdata}<extra></extra>",
        showlegend=False        
    )

    # 5. Layout Customization
    fig.update_layout(
        hovermode='x',
        yaxis_title="Cumulative Return",
        xaxis_title="Date",
        
        hoverlabel=dict(
            bgcolor="white",
            font_size=12,
            font_family="Arial",
            font_color="black",
            bordercolor="#dddddd"
        ),
        
        yaxis=dict(
            tickformat='.1f',
            ticksuffix='%',
            title="Cumulative Returns (%)",
            zeroline=True,
            zerolinewidth=3,
            zerolinecolor="#000000"
        ),

        # --- UPDATED X-AXIS FOR MORE MONTHS ---
        xaxis=dict(
            showspikes=True,
            spikemode='across',
            spikethickness=3,
            spikecolor="red",
            spikedash="dot",
            rangeslider=dict(visible=True),
            type="date",
            
            # Show every month
            dtick="M1",              # Forces a tick for every 1 month
            tickformat="%b %Y",      # Formats as 'Jan 2024'
            tickangle=-90,           # Slanted for better readability than 90 deg
            nticks=24                # Encourages more labels to be drawn
        ),
        
        legend=dict(
            orientation="v", x=1.02, font=dict(size=9),
            itemclick="toggle", itemdoubleclick="toggleothers"
        ),
        updatemenus=[
            dict(
                type="buttons", direction="left", x=0.5, y=1.09, xanchor="center",
                buttons=list([
                    dict(label="Show All", method="restyle", args=[{"visible": [True] * (no_assets + 1)}]),
                    dict(label="Clear All", method="restyle", args=[{"visible": ["legendonly"] * no_assets + [True]}])
                ]),
            )
        ],
        margin=dict(r=150, l=50, b=100, t=120)
    )

    fig.show()

    return

## Plot Annual Dividend yields

In [45]:
# Annual Dividend Yields
import plotly.express as px
import pandas as pd

def plot_annual_dividnd_yields(df:pd.DataFrame):
    asset_names = df.columns.tolist()    
    yield_list = []
    for ticker in asset_names:
        try:
            stock = yf.Ticker(ticker)
            y = stock.info.get('dividendYield', 0) # trailingAnnualDividendYield
            yield_list.append(y if y is not None else 0)
        except:
            yield_list.append(0)
    yields = pd.Series(yield_list, index=asset_names, name='dividendYield') # trailingAnnualDividendYield
    print("Annual Dividend Yields from yf 'dividendYield' (%):")
    display(yields)

    # 1. Convert to percentages and prepare the DataFrame
    # We multiply by 100 because yf.Ticker returns values as decimals (e.g., 0.05 for 5%)
    yields_pct = yields
    plot_df = yields_pct.reset_index()
    plot_df.columns = ['Ticker', 'Yield (%)']

    # 2. Sort the data in descending order for better comparison
    plot_df = plot_df.sort_values(by='Yield (%)', ascending=False)

    # 3. Create the Bar Chart
    fig_yield = px.bar(
        plot_df, 
        x='Ticker', 
        y='Yield (%)', 
        title='<b>Annual Dividend Yields by Asset in %</b>',
        color='Yield (%)', 
        color_continuous_scale='Greens', # Green gradient for higher yields
        text_auto='.2f', # Show yield values on top of bars
        labels={'Yield (%)': 'Dividend Yield (%)'}
    )

    # 4. Refine the layout
    fig_yield.update_layout(
        xaxis_title="Stock Ticker", 
        yaxis_title="Annual Dividend Yield (%)",
        template="plotly_white",
        coloraxis_showscale=False, # Hide the color scale bar for a cleaner look
        margin=dict(l=20, r=20, t=60, b=20),
        height=500
    )

    # Force the x-axis to be categorical so tickers aren't skipped
    fig_yield.update_xaxes(type='category', tickangle=-90)

    fig_yield.show()

    return yields


## Buy & Hold

In [46]:
import plotly.express as px
import pandas as pd
import numpy as np

def buy_and_hold_strategy(cum_returns_df, initial_investment=100000, title_suffix="Assets"):
    """
    Calculates and plots buy-and-hold performance based on a cumulative returns dataframe.
    
    Args:
        cum_returns_df (pd.DataFrame): Dataframe where the last row represents total return (decimal).
        initial_investment (int/float): The starting USD amount.
        title_suffix (str): Description for the chart title (e.g., 'Stocks' or 'Indices').
    """
    
    # 1. Calculation Logic based on Cumulative Returns
    # We take the last row of the cumulative returns (decimal form, e.g., 0.5 for 50%)
    final_cum_returns = cum_returns_df.iloc[-1]/100
    
    # Final Value = Initial Value * (1 + Cumulative Return)
    final_values = initial_investment * (1 + final_cum_returns)
    
    # Margin = Final Value - Initial Value
    margins_usd = final_values - initial_investment
    
    # Yield Percentage
    yields_perc = final_cum_returns * 100

    # 2. Build the Comprehensive Dataframe
    bh_results = pd.DataFrame({
        'Initial_Value_USD': initial_investment,
        'Final_Value_USD': final_values,
        'Margin_USD': margins_usd,
        'Yield_Perc': yields_perc
    }, index=cum_returns_df.columns)

    bh_results.index.name = 'Ticker'

    # 3. Sort by Final Value descending
    bh_results = bh_results.sort_values(by='Final_Value_USD', ascending=False)
    
    # 4. Display the Dataframe for quick audit
    print(f"\n--- Investment Analysis: {title_suffix} ---")
    initial_date = cum_returns_df.index[0].date()
    final_date = cum_returns_df.index[-1].date()
    print(f"From: {initial_date.strftime("%B %d, %Y")} to {final_date.strftime("%B %d, %Y")}")
    print(f"Period: {(final_date - initial_date).days} days")
    # display(bh_results)
    # This ensures the display table uses 0.00 formatting
    display(bh_results.style.format("{:,.2f}"))

    # 5. Prepare for plotting
    plot_df = bh_results.reset_index()

    # 6. Create the Bar Chart
    fig = px.bar(
        plot_df,
        x='Ticker',
        y='Final_Value_USD',
        title=f"Buy and Hold Strategy Performance ${initial_investment:,.0f} Initial Investment in {title_suffix}",
        labels={'Ticker': 'Asset', 'Final_Value_USD': 'Final Value (USD)'},
        template='plotly_white',
        height=700,
        color='Final_Value_USD',
        color_continuous_scale=['#e74c3c', '#f1c40f', '#2ecc71'], 
        color_continuous_midpoint=initial_investment,
        # Custom data for hover
        hover_data=['Initial_Value_USD', 'Margin_USD', 'Yield_Perc']
    )

    # 7. Customize the Hover Template
    # customdata[0] = Initial_Value_USD
    # customdata[1] = Margin_USD
    # customdata[2] = Yield_Perc
    fig.update_traces(
        hovertemplate="""<b>%{x}</b><br><br>""" +
                      """Initial Investment: $%{customdata[0]:,.2f}<br>""" +
                      """Final Value: $%{y:,.2f}<br>""" +
                      """Margin: $%{customdata[1]:,.2f}<br>""" +
                      """Yield: %{customdata[2]:,.2f}%<extra></extra>"""
    )

    # 8. Add the Principal Baseline
    fig.add_hline(
        y=initial_investment, 
        line_dash="dash", 
        line_color="black",
        line_width=2,
        annotation_text="Principal ($100k)", 
        annotation_position="top right"
    )

    # 9. Layout Customization
    fig.update_layout(
        xaxis_tickangle=-90,
        coloraxis_showscale=False,
        yaxis=dict(
            tickformat='$,.0f',
            gridcolor='#eeeeee'
        ),
        hoverlabel=dict(
            bgcolor="white",
            font_size=13,
            font_family="Arial",
            font_color="black",
            bordercolor="#dddddd"
        )
    )

    fig.show()

    return


## Dollar Cost Averaging

In [47]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

def dollar_cost_averaging_strategy(prices_df, monthly_investment=100, title_suffix="Assets"):
    # 1. Ensure Datetime Index
    df_dca = prices_df.copy()
    if not isinstance(df_dca.index, pd.DatetimeIndex):
        df_dca.index = pd.to_datetime(df_dca.index)

    # 2. Resample to get Last Day of each month (Monthly DCA)
    monthly_prices = df_dca.resample('ME').last()
    
    # 3. Calculation Logic
    dca_results = []
    fig = go.Figure()

    # Track Accumulated Investment (Basis) - shared for all calculations
    accumulated_investment = [monthly_investment * (i + 1) for i in range(len(monthly_prices))]
    
    # Calculate for each asset first (so they stay in the background)
    for stock in monthly_prices.columns:
        shares_bought = monthly_investment / monthly_prices[stock]
        total_shares = shares_bought.cumsum()
        investment_value = total_shares * monthly_prices[stock]
        
        # Final Metrics for summary table
        final_val = investment_value.iloc[-1]
        total_invested = accumulated_investment[-1]
        margin = final_val - total_invested
        yield_perc = (margin / total_invested) * 100
        
        dca_results.append({
            'Ticker': stock,
            'Total_Invested': total_invested,
            'Final_Value_USD': final_val,
            'Margin_USD': margin,
            'Yield_Perc_%': yield_perc
        })

        # Add trace with enhanced Hover Data
        # We pass both the Daily Price and the Cumulative Cash to customdata
        fig.add_trace(go.Scatter(
            x=monthly_prices.index, 
            y=investment_value,
            name=stock,
            mode='lines',
            customdata=np.stack((monthly_prices[stock], accumulated_investment), axis=-1),
            hovertemplate=(
                f"<b>{stock}</b><br>" +
                "Date: %{x|%b %d, %Y}<br>" +
                "Stock Price: $%{customdata[0]:,.2f}<br>" +
                "Accumulated Invested: $%{customdata[1]:,.2f}<br>" + # <--- Added this
                "Portfolio Value: $%{y:,.2f}<extra></extra>"
            )
        ))

        # PERMANENT LABEL at the end of the line
        fig.add_annotation(
            x=monthly_prices.index[-1],
            y=investment_value.iloc[-1],
            text=f" {stock}",
            showarrow=False,
            xanchor="left",
            font=dict(size=10, color="black"),
            bgcolor="rgba(255,255,255,0.6)"
        )

    # 4. ADD CASH INVESTED LAST (To keep it in front)
    fig.add_trace(go.Scatter(
        x=monthly_prices.index, 
        y=accumulated_investment,
        name="Cash Invested",
        line=dict(color='black', width=4, dash='dash'), # Width 4 for extra visibility
        hovertemplate="<b>CASH BASIS</b><br>Date: %{x|%b %d, %Y}<br>Total Invested: $%{y:,.2f}<extra></extra>"
    ))

    # 5. Create and Display Summary Table
    summary_df = pd.DataFrame(dca_results).set_index('Ticker')
    summary_df = summary_df.sort_values(by='Final_Value_USD', ascending=False)
    
    print(f"\n--- DCA Investment Analysis: {title_suffix} ---")
    display(summary_df.style.format("{:,.2f}"))

    # 6. Buttons for Toggling
    no_assets = len(monthly_prices.columns)
    # The last trace (index 'no_assets') is the Cash Invested line
    show_all = [True] * (no_assets + 1)
    hide_all = ["legendonly"] * no_assets + [True] # Hide stocks, keep Cash Invested

    fig.update_layout(
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                x=0.5, y=1.15, xanchor="center",
                buttons=list([
                    dict(label="Show All Assets", method="restyle", args=[{"visible": show_all}]),
                    dict(label="Clear All", method="restyle", args=[{"visible": hide_all}])
                ]),
            )
        ]
    )

    # 7. Finalize Layout with RED SPIKE and MONTHLY 90° TICKS
    fig.update_layout(
        title=f"DCA Strategy: Growth of ${monthly_investment}/month in {title_suffix}",
        xaxis_title="Date",
        yaxis_title="Investment Value (USD)",
        height=850,
        template='plotly_white',
        hovermode='x',
        
        hoverlabel=dict(
            bgcolor="white",
            font_size=12,
            font_family="Arial",
            font_color="black",
            bordercolor="#dddddd"
        ),

        xaxis=dict(
            showspikes=True,
            spikemode='across',
            spikethickness=3,
            spikecolor="red",
            spikedash="dot",
            rangeslider=dict(visible=True),
            type="date",
            dtick="M1",              
            tickformat="%b %Y",      
            tickangle=-90,           
            gridcolor='#eeeeee'
        ),
        
        yaxis=dict(tickformat='$,.0f', gridcolor='#eeeeee'),
        margin=dict(r=120, t=150) 
    )
    
    fig.show()
    return


# Correlations and pair Returns

## Plot Heatmap Correlations

In [48]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
from scipy import stats

def plot_interactive_heatmap_correlation(df: pd.DataFrame, triangle: str = 'half'):
    """Plots a masked, interactive heatmap."""
    corr_df = df.corr()
    if triangle == 'half':
        mask = np.triu(np.ones_like(corr_df, dtype=bool), k=0)
        df_masked = corr_df.mask(mask)
    else: 
        df_masked = corr_df

    fig = px.imshow(
        df_masked, text_auto=".2f", color_continuous_scale='RdBu_r', 
        aspect="auto", title=f"Assets Returns Correlation Matrix ({triangle.capitalize()})",
        labels=dict(color="Correlation"), x=corr_df.columns, y=corr_df.columns
    )
    fig.update_layout(height=900, width=950, xaxis_tickangle=-90, template='plotly_white')
    fig.update_traces(hoverongaps=False)
    fig.show()



## Scatter Pair Returns

In [ ]:
def _generate_pair_plots(df, pairs, title):
    """Internal helper to render the grid with regression stats, CIs, and collect summary data."""
    num_pairs = len(pairs)
    
    if num_pairs == 0: 
        print(f"No pairs to plot for: {title}")
        return []

    cols = 3
    rows = int(np.ceil(num_pairs / cols))
    stats_list = []
    
    fig = make_subplots(
        rows=rows, cols=cols, 
        subplot_titles=[f"<b>{a} vs {b}</b><br>(r={df.corr().loc[a,b]:.2f})" for a, b in pairs],
        vertical_spacing=0.1, horizontal_spacing=0.08
    )
    
    for i, (stock_a, stock_b) in enumerate(pairs):
        curr_row = (i // cols) + 1
        curr_col = (i % cols) + 1
        
        clean_data = df[[stock_a, stock_b]].dropna()
        x_vals, y_vals = clean_data[stock_a].values, clean_data[stock_b].values

        # Calculate Regression Stats
        slope, intercept, r_value, p_value, std_err = stats.linregress(x_vals, y_vals)
        r_squared = r_value**2
        
        # Volatility
        vol_a, vol_b = np.std(x_vals), np.std(y_vals)

        stats_list.append({
            'Pair': f"{stock_a} vs {stock_b}",
            'Correlation (r)': r_value,
            'R-Squared': r_squared,
            'p-value': p_value,
            'Slope (Beta)': slope,
            'Std Error': std_err,
            'Intercept (Alpha)': intercept,
            f'Vol_{stock_a}': vol_a,
            f'Vol_{stock_b}': vol_b,
            'Significant?': 'Yes' if p_value < 0.05 else 'No'
        })

        # 1. Scatter Trace
        marker_color = '#3498db' if r_value > 0 else '#e67e22'
        fig.add_trace(
            go.Scatter(x=x_vals, y=y_vals, mode='markers', 
                       marker=dict(size=5, opacity=0.4, color=marker_color), showlegend=False),
            row=curr_row, col=curr_col
        )

        # 2. Regression Line & 95% Confidence Interval Calculation
        x_range = np.linspace(x_vals.min(), x_vals.max(), 100)
        y_pred = intercept + slope * x_range
        
        # CI calculation
        n = len(x_vals)
        dof = n - 2
        t_stat = stats.t.ppf(0.975, dof) # 95% CI
        
        # Standard error of the prediction
        resid = y_vals - (intercept + slope * x_vals)
        s_err = np.sqrt(np.sum(resid**2) / dof)
        
        # Handle division by zero for identical x values
        den = np.sum((x_vals - np.mean(x_vals))**2)
        if den == 0:
            conf_int = np.zeros_like(x_range)
        else:
            conf_int = t_stat * s_err * np.sqrt(1/n + (x_range - np.mean(x_vals))**2 / den)
            
        upper_bound = y_pred + conf_int
        lower_bound = y_pred - conf_int

        # Add CI Shading
        fig.add_trace(
            go.Scatter(
                x=np.concatenate([x_range, x_range[::-1]]),
                y=np.concatenate([upper_bound, lower_bound[::-1]]),
                fill='toself',
                fillcolor='rgba(255, 0, 0, 0.15)',
                line=dict(color='rgba(255, 255, 255, 0)'),
                hoverinfo='skip',
                showlegend=False,
                name='95% CI'
            ),
            row=curr_row, col=curr_col
        )

        # Add Trendline
        fig.add_trace(
            go.Scatter(x=x_range, y=y_pred, mode='lines', line=dict(color='red', width=2), showlegend=False),
            row=curr_row, col=curr_col
        )
        
        # 3. Stats Annotation
        stats_text = f"R²: {r_squared:.3f}<br>p-val: {p_value:.4e}"
        fig.add_annotation(
            dict(xref=f"x{i+1}" if i > 0 else "x", yref=f"y{i+1}" if i > 0 else "y",
                 x=x_vals.min(), y=y_vals.max(), text=stats_text, showarrow=False, align="left",
                 bgcolor="rgba(255, 255, 255, 0.8)", bordercolor="black", borderwidth=1, font=dict(size=10))
        )

    fig.update_layout(height=400 * rows, width=1100, title_text=title, template='plotly_white')
    fig.show()
    return stats_list

def plot_extreme_correlations(df: pd.DataFrame, num_pairs=9):
    corr_matrix = df.corr()
    mask = np.triu(np.ones(corr_matrix.shape), k=0).astype(bool)
    corr_series = corr_matrix.where(~mask).stack()

    # Get pairs
    pos_pairs = corr_series.sort_values(ascending=False).head(num_pairs).index.tolist()
    neg_series = corr_series[corr_series < 0]
    neg_pairs = neg_series.sort_values(ascending=True).head(num_pairs).index.tolist()

    # Generate Plots
    pos_stats = _generate_pair_plots(df, pos_pairs, "Top Positive Correlations (with 95% CI)")
    neg_stats = _generate_pair_plots(df, neg_pairs, "Top Negative Correlations (with 95% CI)")

    all_stats = pos_stats + neg_stats

    # --- SAFETY CHECK 1: Ensure we actually have data ---
    if not all_stats:
        print("⚠️ No correlation pairs found to display in the summary table.")
        return

    summary_df = pd.DataFrame(all_stats).set_index('Pair')
    
    # --- SAFETY CHECK 2: Verify the column exists before sorting ---
    if 'R-Squared' not in summary_df.columns:
        print(f"⚠️ Column 'R-Squared' missing. Available: {summary_df.columns.tolist()}")
        # Display raw data so you can see what's inside
        display(summary_df)
        return
    
    print("\n" + "="*80)
    print("REGRESSION & VOLATILITY SUMMARY TABLE")
    print("="*80)

    # Sort by R-Squared, then set the index just for the display
    styled_df = (summary_df
                 .sort_values('R-Squared', ascending=False)
                 .style.format(precision=4))
    
    display(styled_df)

    # display(summary_df.sort_values('R-Squared', ascending=False).set_index('Pair').style.format(precision=4))

    # --- ADD INTERPRETATION NOTES BELOW TABLE ---
    print("\n" + "-"*50)
    print("INTERPRETATION GUIDE:")
    print("-"*50)
    notes = (
        "• p-value Significance: Probability the correlation happened by chance. \n"
        "  Small values (e.g., < 1.42e-05) mean the relationship is mathematically solid.\n"
        "• Significance?: 'Yes' if p_value < 0.05.\n"
        "• Beta (Slope): The magnitude. A slope of 1.2 means for every 1% change in Stock A, \n"
        "  Stock B tends to move 1.2%.\n"
        "• Alpha (Intercept): The 'excess' return of Stock B if Stock A's return was zero.\n"
        "• Low Standard Error: Relationship is tight; Beta is a reliable predictor.\n"
        "• High Standard Error: Lots of 'noise'; Beta is less reliable for hedging/pairs trading.\n"
        "• Volatility (Vol):\n"
        "  - High Vol + High Corr: High-octane comovers.\n"
        "  - Low Vol + High Corr: Stable 'pairs trading' candidates.\n"
        "  - Interpretation: If Vol Stock B > Vol Stock A, Beta will naturally be higher. \n"
        "    Check this to see if Beta is driven by correlation or just swing scale."
    )
    print(notes)


# Covariance Variance

## Covariance Heatmap Plot

In [50]:
import plotly.express as px
import pandas as pd
import numpy as np

def plot_annualized_covariance_heatmap(df: pd.DataFrame, triangle: str = 'half'):
    """
    Calculates the Annualized Covariance matrix (Daily Cov * 252) 
    and plots a heatmap including the Variance on the diagonal.
    """
    # 1. Calculate Daily Covariance and Annualize it
    # Annual Cov = Daily Cov * 252 (trading days in a year)
    annual_cov = df.cov() * 252
    
    # 2. Masking Logic
    if triangle == 'half':
        # k=1 keeps the diagonal (Variance) visible while hiding the upper mirror
        mask = np.triu(np.ones_like(annual_cov, dtype=bool), k=1)
        plot_data = annual_cov.mask(mask)
    else:
        plot_data = annual_cov

    # 3. Create Heatmap
    # Using .4f precision as annualized numbers are larger
    fig = px.imshow(
        plot_data,
        text_auto=".4f", 
        color_continuous_scale='Viridis',
        aspect="auto",
        title=f"Annualized Covariance & Variance Matrix ({triangle.capitalize()})",
        labels=dict(color="Annual Cov/Var"),
        x=annual_cov.columns,
        y=annual_cov.columns
    )

    # 4. Layout
    fig.update_layout(
        height=900,
        width=1000,
        xaxis_tickangle=-90,
        template='plotly_white',
        # Annotate that diagonal = Variance
        annotations=[dict(
            x=1.1, y=1.05, xref="paper", yref="paper",
            text="Note: Diagonal values represent Asset Variance (σ²), the sqrt will be the Annualized Volatility (σ) or stdDev ",
            showarrow=False, font=dict(size=12, color="gray")
        )]
    )
    
    fig.update_traces(hoverongaps=False)
    fig.show()


# # Example usage of the function

# # Create a sample DataFrame similar to your `daily_returns`
# np.random.seed(0)
# daily_returns = pd.DataFrame(
#     np.random.randn(100, 6),
#     columns=[f'Stock_{i}' for i in range(1, 7)]
# )

# covariance = daily_returns.corr()

# plot_annualized_covariance_heatmap(covariance)

# Efficient Frontier (Sharpe Ratio)

## GUI interface

In [51]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.optimize import minimize
from IPython.display import display, clear_output
import ipywidgets as widgets
import yfinance as yf
from plotly.graph_objects import FigureWidget


def get_portfolio_stats(weights, mean_ann, cov_ann, rf, yields=None):
    weights = np.array(weights)
    total_mu = mean_ann + (yields if yields is not None else 0)
    mu = np.sum(weights * total_mu)
    vol = np.sqrt(weights.T @ cov_ann @ weights)
    sr = (mu - rf) / vol if vol > 0 else 0
    return mu, vol, sr

# --- CLEANUP BLOCK ---
try:
    out.close()
except:
    pass

def run_full_frontier_analysis(
    rets, curr_port_weights, curr_port_vol, curr_port_ret, 
    mean_ann, cov_ann, rf_default=0.045, portfolio_value=100000, yields=None,
    long_only=True, n_sims=2000, seed=123, maintenance_margin=0.25 
):
    
    global out
    n_assets = len(mean_ann)
    asset_names = rets.columns.tolist()
    
    # --- BUSY FLAG (Stops triple-printing) ---
    is_busy = False 

    # --- PRE-CALCULATE EFFICIENT FRONTIER ---
    bounds = [(0, 1) for _ in range(n_assets)] if long_only else [(-1.0, 1.0) for _ in range(n_assets)]
    cons_sum1 = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0}

    res_var_init = minimize(lambda w: get_portfolio_stats(w, mean_ann, cov_ann, rf_default, yields)[1], n_assets*[1./n_assets], bounds=bounds, constraints=[cons_sum1])
    mu_min_var_curve = get_portfolio_stats(res_var_init.x, mean_ann, cov_ann, rf_default, yields)[0]
    mu_max_ret_curve = (mean_ann + yields).max()
    
    target_range = np.linspace(mu_min_var_curve, mu_max_ret_curve, 40)
    frontier_vols = []
    for r in target_range:
        res = minimize(lambda w: get_portfolio_stats(w, mean_ann, cov_ann, rf_default, yields)[1], n_assets*[1./n_assets], 
                       bounds=bounds, constraints=[cons_sum1, {'type': 'eq', 'fun': lambda w: get_portfolio_stats(w, mean_ann, cov_ann, rf_default, yields)[0] - r}])
        frontier_vols.append(res.fun)

    # --- UI COMPONENTS ---

    # Create a common style and layout to keep it clean
    long_desc_style = {'description_width': 'initial'}
    wide_layout = widgets.Layout(width='350px')

    portfolio_value_input = widgets.FloatText(description="Portfolio Value (V): $", value=round(portfolio_value, 2), style=long_desc_style, layout=wide_layout)
    target_mode = widgets.Dropdown(options=[('Manual Input', 'manual'), ('Risk Slider', 'slider')], value='slider', description='Target Mode:', style=long_desc_style, layout=wide_layout)
    target_val_type = widgets.Dropdown(options=[('Risk %', 'risk'), ('Return %', 'ret')], value='risk', description='Manual Option:', style=long_desc_style, layout=wide_layout)
    target_input = widgets.FloatText(description="Value %:", value=round((curr_port_vol) * 100, 0), style=long_desc_style, layout=wide_layout)
    risk_slider = widgets.IntSlider(value=50, min=0, max=100, step=1, description='Risk Budget %:', continuous_update=False, style=long_desc_style, layout=wide_layout)
    
    rf_input = widgets.FloatText(description="Risk Free Rate %:", value=round(rf_default * 100, 2), style=long_desc_style, layout=wide_layout)
    margin_input = widgets.FloatText(description="Margin Rate %:", value=5.0, style=long_desc_style, layout=wide_layout) 

    range_info = widgets.Label(value="Initializing range...", style=long_desc_style, layout=widgets.Layout(width='800px'))
    status_label = widgets.Label(value="Target Volatility: --", style=long_desc_style, layout=wide_layout)
    compare_to = widgets.Dropdown(
        options=['Max Sharpe', 'Min Variance', 'Max Return', 'Desired Portfolio', 'Desired CML Portfolio', 'CML 50-50 Port', 'CML 30-70 Port', 'CML -30/+130 Port'],
        value='Max Sharpe', description='Rebalance To:', style=long_desc_style, layout=wide_layout
    )
    out = widgets.Output()

    def toggle_inputs(change):
        if target_mode.value == 'manual':
            target_val_type.disabled, target_input.disabled, risk_slider.disabled = False, False, True
        else:
            target_val_type.disabled, target_input.disabled, risk_slider.disabled = True, True, False

    # --- REACTIVE UPDATE FUNCTION ---
    def update_dashboard(*args):
        nonlocal is_busy
        if is_busy: return 
        is_busy = True
        
        rf_current = rf_input.value / 100.0
        margin_rate = margin_input.value / 100.0
        
        with out:
            clear_output(wait=True)
            display(widgets.HTML(f"<h2 style='color:#2e7d32;'>Portfolio Analysis. Equity: ${portfolio_value_input.value:,.2f}</h2>"))
            
            res_sr = minimize(lambda w: -get_portfolio_stats(w, mean_ann, cov_ann, rf_current, yields)[2], n_assets*[1./n_assets], bounds=bounds, constraints=[cons_sum1])
            w_max_sr = res_sr.x
            mu_max_sr, vol_max_sr, sr_max_sr = get_portfolio_stats(w_max_sr, mean_ann, cov_ann, rf_current, yields)
            
            vol_min_var, mu_min_var = frontier_vols[0], target_range[0]
            vol_max_ret, mu_max_ret = frontier_vols[-1], target_range[-1]
            w_max_ret_port = np.zeros(n_assets); w_max_ret_port[np.argmax((mean_ann + yields).values)] = 1.0


            # --- TARGET LOGIC WITH SAFETY GUARDRAIL ---
            # Define Safety Buffer: 30% instead of legal 25% to be safe
            safe_maintenance = maintenance_margin + 0.05
            
            # Calculate the maximum buying power before hitting a margin call immediately
            # Buying Power = Equity / Maintenance
            max_buying_power = portfolio_value_input.value / safe_maintenance
            max_leverage_ratio = max_buying_power / portfolio_value_input.value
            
            # The maximum volatility we can safely target using CML
            max_safe_vol = vol_max_sr * max_leverage_ratio

            # Ceiling is now the LESSER of the Max Return asset or the Safety Limit
            max_allowed_risk = min(vol_max_ret, max_safe_vol)
            
            # Target Value (t_v) and Manual / Slider Mode selector
            if target_mode.value == 'slider':
                t_m = 'risk'

                t_v = vol_min_var + (risk_slider.value / 100.0) * (max_allowed_risk - vol_min_var)
                
                range_info.value = f"Safety Cap: {max_allowed_risk:.2%} Volatility | Max Leverage: {max_leverage_ratio:.2f}x before Margin Call (we have an extra 5% safety margin since MC is at 25%)"
                status_label.value = f"Risk Budget Selected: {risk_slider.value}% | Target Volatility: {t_v:.2%}"
            else:
                # Manual Input Mode
                t_m = target_val_type.value
                t_v = target_input.value / 100.
                
                # Warning if manual input exceeds safety
                if t_m == 'risk' and t_v > max_safe_vol:
                    range_info.value = f"⚠️ WARNING: Target risk exceeds immediate Margin Call limits that is {max_allowed_risk:.2%}"
                else:
                    range_info.value = f"Safety Cap: {max_allowed_risk:.2%} Volatility | Max Leverage: {max_leverage_ratio:.2f}x before Margin CAll (we have an extra 5% safety margin since MC is at 25%)"
                status_label.value = f"Manual Target {'Return' if t_m == 'ret' else 'Risk'} Set → Target: {t_v:.2%}"

            if t_m == 'ret':
                res_d = minimize(lambda w: get_portfolio_stats(w, mean_ann, cov_ann, rf_current, yields)[1], n_assets*[1./n_assets], bounds=bounds, constraints=[cons_sum1, {'type': 'eq', 'fun': lambda w: get_portfolio_stats(w, mean_ann, cov_ann, rf_current, yields)[0] - t_v}])
            else:
                # If user sets a target risk (vol), find the portfolio with MAXIMUM return for that risk
                res_d = minimize(lambda w: -get_portfolio_stats(w, mean_ann, cov_ann, rf_current, yields)[0], n_assets*[1./n_assets], bounds=bounds, constraints=[cons_sum1, {'type': 'ineq', 'fun': lambda w: t_v - get_portfolio_stats(w, mean_ann, cov_ann, rf_current, yields)[1]}])

            w_des = res_d.x if res_d.success else w_max_sr
            d_st = get_portfolio_stats(w_des, mean_ann, cov_ann, rf_current, yields)


            # --- UPDATED KINKED CML COORDINATES ---
            # Volatility is always the target volatility
            vol_des_cml = t_v
            
            # Determine leverage multiplier (L) relative to the Tangency Portfolio
            w_cml_lever = t_v / vol_max_sr if vol_max_sr > 0 else 0
            
            if t_v <= vol_max_sr:
                # LENDING ZONE (Green Line): Linear blend of Cash and Max Sharpe
                mu_des_cml = rf_current + w_cml_lever * (mu_max_sr - rf_current)
            else:
                # BORROWING ZONE (Red Line): Leveraged Max Sharpe minus Margin Cost
                # Formula: mu = mu_max_sr + (Leverage - 1) * (mu_max_sr - margin_rate)
                mu_des_cml = mu_max_sr + (w_cml_lever - 1) * (mu_max_sr - margin_rate)
            
            # Asset weights for the CML portfolio
            w_des_cml = w_cml_lever * w_max_sr




            # --- PLOTTING ---
            rng = np.random.default_rng(seed)
            # Generating simulated weights for the scatter cloud
            sim_weights = rng.dirichlet(np.ones(n_assets), n_sims)
            sim_res = np.array([get_portfolio_stats(w, mean_ann, cov_ann, rf_current, yields) for w in sim_weights])

            fig = go.Figure()

            # POINTS: Monte Carlo Simulations
            fig.add_trace(go.Scatter(x=sim_res[:,1], y=sim_res[:,0], mode='markers', marker=dict(color=sim_res[:,2], colorscale='Viridis', showscale=True), name='Simulations', opacity=0.5))
            
            # LINE: Efficient Frontier
            fig.add_trace(go.Scatter(x=frontier_vols, y=target_range, mode='lines', line=dict(color='black', width=4), name='Efficient Frontier'))
            fig.add_annotation(
                x=vol_max_ret*0.9, y=mu_max_ret,
                text=f"Efficient Frontier",
                showarrow=True, arrowhead=2, ax=0, ay=-40, bgcolor="white", bordercolor="black"
            )
            
            # LINE: CML (Ideal Capital Market Line based on Risk-free rate (Rf))
            max_x_cml = max(max(frontier_vols), vol_des_cml) * 1.0
            fig.add_trace(go.Scatter(x=[0, max_x_cml], y=[rf_current, rf_current + sr_max_sr*max_x_cml], mode='lines', line=dict(color='green', dash='dash', width=2), name='Ideal CML (RF Rate)'))
            fig.add_annotation(
                x=max_x_cml, y=rf_current + sr_max_sr*max_x_cml,
                text=f"Ideal CML (RF Rate)",
                showarrow=True, arrowhead=2, ax=-20, ay=-30, bgcolor="white", bordercolor="green"
            )
            
            # --- SECONDARY LINE: BORROWING LINE (KINKED CML) ---
            # Slope of the borrowing line is determined by the Margin Rate, not the RF Rate
            # Borrowing Slope = (Portfolio Return - Margin Rate) / Portfolio Volatility
            borrowing_sr = (mu_max_sr - margin_rate) / vol_max_sr

            # The line starts at Max Sharpe and extends to the right edge of the plot
            fig.add_trace(go.Scatter(
                x=[vol_max_sr, max_x_cml], 
                y=[mu_max_sr, mu_max_sr + borrowing_sr * (max_x_cml - vol_max_sr)], 
                mode='lines',
                line=dict(color='red', dash='dot', width=2), 
                name='Borrowing Line (Margin rate)'
            ))
            fig.add_annotation(
                x=max_x_cml, y=mu_max_sr + borrowing_sr * (max_x_cml - vol_max_sr),
                text=f"Borrowing Line (Margin rate)",
                showarrow=True, arrowhead=2, ax=-10, ay=60, bgcolor="white", bordercolor="red"
            )

            # 3. SPREAD Cost Annotation (Vertical gap at Desired Volatility)
            if vol_des_cml > vol_max_sr:
                ideal_ret = rf_current + sr_max_sr * vol_des_cml
                actual_ret = mu_max_sr + borrowing_sr * (vol_des_cml - vol_max_sr)
                spread_loss_usd = (ideal_ret - actual_ret) * portfolio_value_input.value

                fig.add_annotation(
                    x=vol_des_cml, y=(ideal_ret + actual_ret) / 2,
                    text=f"Margin vs Risk-free Spread Cost:<br>${spread_loss_usd:,.2f}/yr",
                    showarrow=True, arrowhead=2, ax=60, ay=-60, font=dict(size=10)
                )

            # --- POINT: CASH FLOW BREAKEVEN POINT ---
            # Leverage (L) where Dividends = Interest
            # L * Portfolio_Yield = (L - 1) * Margin_Rate
            # Solving for L: L = Margin_Rate / (Margin_Rate - Portfolio_Yield)
            port_yield = np.sum(w_max_sr * yields)
            if margin_rate > port_yield:
                breakeven_lev = margin_rate / (margin_rate - port_yield)
                # Ensure the breakeven point is within a realistic leverage range
                if 1.0 < breakeven_lev < max_leverage_ratio: #3.0:
                    be_vol = breakeven_lev * vol_max_sr
                    be_ret = mu_max_sr + borrowing_sr * (be_vol - vol_max_sr)
                    
                    fig.add_trace(go.Scatter(
                        x=[be_vol], y=[be_ret],
                        mode='markers+text',
                        text=["Cash Flow Breakeven"],
                        textposition="top right",
                        marker=dict(color='black', size=10, symbol='x'),
                        name='Break even: Div Income = Margin Cost'
                    ))
                    
                    fig.add_annotation(
                        x=be_vol, y=be_ret,
                        text="Point where dividends<br>cover all interest",
                        showarrow=True, ax=0, ay=-60, font=dict(size=10)
                    )

            # POINTS: CML portfolios
            cml_map = {"CML 50-50 Port": 0.5, "CML 30-70 Port": 0.7, "CML -30/+130 Port": 1.3}
            for lab, w_r in cml_map.items():
                fig.add_trace(go.Scatter(x=[w_r * vol_max_sr], y=[(1-w_r)*rf_current + w_r*mu_max_sr], mode='markers+text', text=[lab], textposition="bottom center", marker=dict(color='orange', size=12, symbol='diamond', line=dict(width=2, color='black')), name=lab))

            # Points & Markers 
            pts = [(vol_max_ret, mu_max_ret, 'Max Return', 'green', 'circle'), (vol_max_sr, mu_max_sr, 'Max Sharpe', 'red', 'star'), (vol_min_var, mu_min_var, 'Min Variance', 'blue', 'circle'), (curr_port_vol, curr_port_ret, 'Current', 'purple', 'square'), (d_st[1], d_st[0], 'Desired Portfolio', 'gold', 'hexagram'), (vol_des_cml, mu_des_cml, 'Desired CML Portfolio', 'cyan', 'diamond'), (0, rf_current, 'Risk Free', 'black', 'circle-open')]
            for v, r, name, col, sym in pts:
                fig.add_trace(go.Scatter(x=[v], y=[r], mode='markers+text', text=[name], textposition="bottom center", marker=dict(color=col, size=14, symbol=sym, line=dict(width=2, color='black')), name=name))
            
            fig.update_layout(title=f"Dashboard | RF: {rf_current:.2%} | Margin: {margin_rate:.2%}", template="plotly_white", height=600, xaxis_title="Volatility (%)  -Total risk: systematic & un-systematic", yaxis_title="Return (%)", legend=dict(orientation="h", y=-0.15))


            # fig.show()
            # Convert the fig.show() to a Widget that can listen for mouse clicks
            # --- REPLACE FIG.SHOW() WITH THIS INTERACTIVE BLOCK ---

            # --- INTERACTIVE CLICK LOGIC ---
            # 1. Force the layout to be larger
            fig.update_layout(
                height=800, # Increased height in pixels
                width=1200, # Optional: uncomment if you want a fixed wide view
                autosize=True,
                margin=dict(l=10, r=10, t=50, b=10)
            )
            # 2. Create the widget
            fig_widget = FigureWidget(fig)
            # This function runs every time you click the plot
            def on_click(trace, points, state):
                if not points.point_inds: return
                # Get the Volatility (x) of where the user clicked
                new_vol = points.xs[0]
                # Force the dashboard into Manual/Risk mode
                # Update UI controls to match the click
                # This will automatically trigger update_dashboard via the observers
                target_mode.value = 'manual'
                target_val_type.value = 'risk'
                
                # Update the input box (which triggers update_dashboard automatically)
                target_input.value = round(new_vol * 100, 2)
            # Attach the click event to the simulation cloud and frontier line,
            fig_widget.data[0].on_click(on_click) # Simulation Cloud
            fig_widget.data[1].on_click(on_click) # Frontier Line
            fig_widget.data[2].on_click(on_click) # CML Line
            fig_widget.data[3].on_click(on_click) # Borrowing Line
            fig_widget.data[4].on_click(on_click) # spread
            fig_widget.data[5].on_click(on_click) # Break even point Dividends = Interests
            fig_widget.data[6].on_click(on_click) # Borrowing Line
            fig_widget.data[7].on_click(on_click) # Points CML portfolios
            fig_widget.data[8].on_click(on_click) # Portfolio markers (Max ret, Min var, Max SR)

            display(fig_widget)


            # --- TABLES ---
            all_w = {'Current': curr_port_weights, 'Max Sharpe': w_max_sr, 'Min Variance': res_var_init.x, 'Max Return': w_max_ret_port, 'Desired Portfolio': w_des, 'Desired CML Portfolio': w_des_cml}
            cml_map = {"CML 50-50 Port": 0.5, "CML 30-70 Port": 0.7, "CML -30/+130 Port": 1.3}
            for lab, mult in cml_map.items(): all_w[lab] = mult * w_max_sr

            t_name = compare_to.value
            rebal_df = pd.DataFrame(index=asset_names + ['Risk-Free Asset'])
            for s_name, weights in all_w.items():
                rebal_df.loc[asset_names, s_name] = weights
                # rebal_df.loc['Risk-Free Asset', s_name] = 1.0 - np.sum(weights)

                cash_val = 1.0 - np.sum(weights)
                # Fixing floating point. If the value is smaller than 0.000001, force it to exactly 0
                rebal_df.loc['Risk-Free Asset', s_name] = cash_val if abs(cash_val) > 1e-6 else 0.0

            rebal_df.loc['TOTAL', :] = rebal_df.sum()
            asset_rows = asset_names + ['Risk-Free Asset']
            rebal_df['Action (Delta)'] = rebal_df[t_name] - rebal_df['Current']
            rebal_df['Action (USD)'] = rebal_df['Action (Delta)'] * portfolio_value_input.value
            rebal_df.loc['TOTAL', ['Action (Delta)', 'Action (USD)']] = 0.0


            # --- BREAKDOWN BOX ---
            stocks_val = rebal_df.loc[asset_names, t_name].sum() * portfolio_value_input.value            
            loan_val = abs(rebal_df.loc['Risk-Free Asset', t_name] * portfolio_value_input.value) if rebal_df.loc['Risk-Free Asset', t_name] < 0 else 0
            # price_drop_mc: % drop from current value (md: margin call)
            price_drop_mc = 1 - (loan_val / (1 - maintenance_margin)) / stocks_val if loan_val > 0 else 0
            # critical_value_usd: The actual market value in dollars that triggers the call
            critical_value_usd = loan_val / (1 - maintenance_margin) if loan_val > 0 else 0
            
            mc_html = f"<br><b>⚠️ Margin Call at Price Drop (happens when E = V*MR, MR:Maintenance Requirement~25% -varies for each stock):</b> {price_drop_mc:.2%} (Portfolio Value: ${critical_value_usd:,.2f})" if loan_val > 0 else ""
            display(widgets.HTML(f"""<div style='background-color:#f1f8e9; padding:15px; border-radius:10px; border:1px solid #2e7d32; margin-bottom:10px;'>
                                 <h3>Breakdown: {t_name}</h3>
                                 <b>Total Stock Exposure (Holdings) Assets(V) = Liabilities(D) + Equity(E):  </b> ${stocks_val:,.2f} ({rebal_df.loc[asset_names, t_name].sum():.2%})<br>
                                 <b>Risk-Free (Cash/Margin or Debt(D)):</b> ${rebal_df.loc['Risk-Free Asset', t_name] * portfolio_value_input.value:,.2f} ({rebal_df.loc['Risk-Free Asset', t_name]:.2%}){mc_html}<br>
                                 <b>Net Liquidation Value (Equity = V - D):</b> ${portfolio_value_input.value:,.2f}</div>"""))

            # --- REBALANCE TABLE WITH TRANSPARENCY ---
            print("\nREBALANCE TABLE")
            f_dict = {col: '{:.2%}' for col in all_w.keys()}; f_dict.update({'Action (Delta)':'{:+.2%}', 'Action (USD)':'${:+,.2f}'})
            display(rebal_df.style.format(f_dict)
                .bar(subset=(asset_rows, 'Action (Delta)'), align='mid', color=['#ff9999', '#99ff99'])
                .background_gradient(subset=(asset_rows, t_name), cmap='BuGn')
                .map(lambda x: 'background-color: transparent; background-image: none;' if abs(x) < 1e-6 else '', subset=(asset_rows, t_name))
                .set_properties(**{'background-color': '#ffebee', 'color': '#c62828', 'font-weight': 'bold'}, subset=pd.IndexSlice[['Risk-Free Asset'], [c for c in all_w.keys() if rebal_df.loc['Risk-Free Asset', c] < -1e-6]])
                .set_properties(**{'font-weight': 'bold', 'border-top': '2px solid black'}, subset=pd.IndexSlice[['TOTAL'], :]))


            # --- PERFORMANCE TABLE ---
            stats_list = []
            for k, w in all_w.items():
                if "CML" in k:
                    mult = w_cml_lever if k == 'Desired CML Portfolio' else cml_map[k]
                    m, v = (1-mult)*rf_current + mult*mu_max_sr, mult*vol_max_sr
                    p_y = mult * np.sum(w_max_sr * yields)
                elif k == 'Current': m, v, p_y = curr_port_ret, curr_port_vol, np.sum(curr_port_weights*yields)
                else: m, v, _ = get_portfolio_stats(w, mean_ann, cov_ann, rf_current, yields); p_y = np.sum(w*yields)
                
                cash_pos = rebal_df.loc['Risk-Free Asset', k]
                
                interest_cost_ann = abs(cash_pos) * margin_rate if cash_pos < -1e-6 else 0
                final_ret = m - interest_cost_ann
                drop_mc = 1 - (abs(cash_pos) / (1 - maintenance_margin)) / rebal_df.loc[asset_names, k].sum() if cash_pos < 0 else np.nan
                
                # Monthly Cash Flow Math
                monthly_div = (p_y * portfolio_value_input.value) / 12
                monthly_interest = (interest_cost_ann * portfolio_value_input.value) / 12
                net_monthly_cash = monthly_div - monthly_interest                
                
                stats_list.append({
                    'Portfolio': k, 
                    'Return (%)': final_ret*100, 
                    'Volatility (%)': v*100, 
                    'Sharpe': (final_ret - rf_current)/v if v > 0 else 0, 
                    'Div Yield (%) (annual)': p_y*100,
                    'Margin Rate (%)': margin_rate,
                    'Margin Cost (usd) (annual)': interest_cost_ann * portfolio_value_input.value,
                    'Est. Monthly Div Income (usd)': monthly_div, 
                    'Est. Monthly Interest (usd)': monthly_interest,
                    'Net Monthly Cash (usd)': net_monthly_cash, 
                    'Drop to Margin Call (%)': drop_mc * 100,
                    'Break-Even Margin Rate (%)': (m - rf_current) / abs(cash_pos) * 100 if cash_pos < -1e-6 else np.nan
                })
            
            comp_df = pd.DataFrame(stats_list).set_index('Portfolio').sort_values(by=['Sharpe', 'Volatility (%)'], ascending=[False, True])
            print("\nSTRATEGY PERFORMANCE & INCOME COMPARISON")
            display(comp_df.style.format("{:,.2f}%", subset=['Return (%)', 'Volatility (%)', 'Div Yield (%) (annual)', 'Drop to Margin Call (%)', 'Break-Even Margin Rate (%)', 'Margin Rate (%)'])
                    .format({'Margin Cost (usd) (annual)':"${:,.2f}", 'Est. Monthly Interest (usd)':"${:,.2f}", 'Net Monthly Cash (usd)':"${:,.2f}", 'Est. Monthly Div Income (usd)':"${:,.2f}"})
                    .format({'Sharpe': '{:.3f}'})
                    .background_gradient(cmap='RdYlGn', subset=['Sharpe'])
                    .background_gradient(cmap='Greens', subset=['Net Monthly Cash (usd)'], vmin=0)
                    .map(lambda v: 'background-color: transparent; color: red;' if v < 0 else '', subset=['Net Monthly Cash (usd)'])
                    .map(lambda v: 'background-color: transparent; color: red;' if v < 0 else '', subset=['Drop to Margin Call (%)']))


            # --- SENSITIVITY TABLE ---
            # 1. Calculate the Break-Even Rate for the Desired CML Portfolio
            # We strip spaces because the index might have them from the deduplication above
            row_key = [idx for idx in comp_df.index if idx.strip() == 'Desired CML Portfolio'][0]
            be_rate_raw = comp_df.loc[row_key, 'Break-Even Margin Rate (%)']
            desiredCmlBreakEven_rate = round(be_rate_raw / 100, 4) if not np.isnan(be_rate_raw) else margin_rate
            
            # 2. Build the list of rates and REMOVE DUPLICATES to prevent KeyError
            s_rates_raw = [0.03, 0.05, 0.07, 0.09, 0.12, desiredCmlBreakEven_rate, desiredCmlBreakEven_rate + 0.05]
            # set() removes duplicates, sorted() puts them in order
            s_rates = sorted(list(set(s_rates_raw)))
            
            sens_data = []
            for r_val in s_rates:
                cml_cash = 1.0 - np.sum(w_des_cml)
                c_cost = abs(cml_cash) * r_val if cml_cash < -1e-6 else 0
                net_r = mu_des_cml - c_cost
                sens_data.append({
                    'Loan Rate (%)': r_val * 100, 
                    'Volatility (%)': vol_des_cml * 100, 
                    'Net Return (%)': net_r * 100, 
                    'Net Sharpe': (net_r - rf_current) / vol_des_cml if vol_des_cml > 0 else 0
                })
            display(widgets.HTML("<h4 style='color:#ef6c00;'>🔍 Margin Rate Sensitivity (<span style='color:black; font-weight:bold;'>Desired CML Port</span>)</h4>"))
            # index is now guaranteed unique due to the sorted(list(set())) logic
            display(pd.DataFrame(sens_data).set_index('Loan Rate (%)').style.format("{:.2f}%", subset=['Net Return (%)', 'Volatility (%)']).format({'Net Sharpe': '{:.3f}'})
                    .map(lambda v: 'background-color: transparent; color: green; font-weight: bold;' if v > 0 else '', subset=['Net Sharpe'])
                    .map(lambda v: 'background-color: transparent; color: red; font-weight: bold;' if v <= 0 else '', subset=['Net Sharpe']))



            # --- EDUCATIONAL NOTES & DISCLOSURES ---
            display(widgets.HTML(f"""<div style='background-color:#e3f2fd; padding:15px; border-radius:10px; border:1px solid #1976d2; margin-top:20px;'>
            <h4 style='margin-top:0; color:#0d47a1;'>📘 Financial Logic & Column Definitions</h4>
            <ul style='padding-left: 20px;'>
                <li><b>Risk-Free Rate:</b> The yield earned on idle (positvie) cash (e.g. 1-Yr Treasury). Applied when cash is positive.</li>
                <li><b>Net Monthly Cash:</b> Monthly Dividends minus Robinhood Interest.</li>
                <li><b>Margin Rate:</b> The cost paid to borrow funds (Default: 5%). Applied when cash is negative.</li>
                <li><b>Break-Even Margin Rate:</b> The maximum interest rate you can pay before leverage becomes worse than just holding the Max Sharpe portfolio without borrowing.</li>
                <li><b>CML 50-50 Port:</b> You are "Lending." You invest 50% in the Max Sharpe stocks and 50% in Risk-Free assets (Treasuries) earning {rf_input.value}%.</li>
                <li><b>CML -30/+130 Port:</b> You are "Borrowing." You invest 130% of your net worth in stocks by taking a 30% Margin Loan at {margin_input.value}% of margin rate.</li>
                <li><b>Return (%):</b> This is the <b>Net Annual Return</b>. If you are leveraged (Borrowing), we have automatically subtracted the cost of the Robinhood loan (Margin Costs), or add Risk-Free yields if you have cash</li>
                <li><b>Drop to Margin Call (MC): The market crash percentage (e.g., 69.23%) that triggers a Margin Call based on {maintenance_margin:.0%} maintenance.</b> 
                    <ul>
                        <li><b style='color:#555;'>nan:</b> You have extra cash (No risk of margin call). You have positive cash. You cannot get a margin call.</li>
                        <li><b>100%:</b> You have very little debt; stocks must hit zero to trigger a call.</li>
                        <li><b>< 100%:</b> The percentage your stocks can fall before your equity hits the {maintenance_margin:.0%} maintenance requirement. 
                        <i>Example: If it shows 70%, a 70% market crash triggers a forced liquidation.</i></li>
                    </ul>
                </li>
            </ul>
            </div>"""))

            # --- UPDATED EDUCATIONAL NOTES FOR BREAK-EVEN LOGIC ---
            display(widgets.HTML(f"""<div style='background-color:#e3f2fd; padding:15px; border-radius:10px; border:1px solid #1976d2; margin-top:20px;'>
            <h4 style='margin-top:0; color:#0d47a1;'>📘 Understanding Break-Even Margin Rates</h4>
            <ul style='padding-left: 20px;'>
                <li><b>Why is the Break-Even Rate 100%+?</b> This happens when your stocks have a very high Sharpe Ratio. It indicates that the <i>Risk Premium</i> you are capturing is so high that even very expensive debt would still be mathematically "efficient."</li>
                <li><b>How to use it:</b> Compare this to your <b>Margin Rate (5%)</b>. If the Break-Even is 158%, you are in a very safe "Interest Spread" zone. Leverage is highly beneficial here.</li>
                <li><b>Net Monthly Cash:</b> Note that <b>CML -30/+130</b> produces <b>${comp_df.loc['CML -30/+130 Port', 'Net Monthly Cash (usd)']:,.2f}</b> net. This accounts for the <b>${comp_df.loc['CML -30/+130 Port', 'Est. Monthly Interest (usd)']:,.2f}</b> interest paid to Robinhood.</li>
                <li><b>Drop to Margin Call:</b> For the 130% portfolio, a <b>{comp_df.loc['CML -30/+130 Port', 'Drop to Margin Call (%)']:.2f}%</b> drop in your stocks triggers a margin call.</li>
            </ul>
            </div>"""))
            
            # --- UPDATED EDUCATIONAL NOTES FOR NEGATIVE MC VALUES ---
            display(widgets.HTML(f"""<div style='background-color:#ffebee; padding:15px; border-radius:10px; border:1px solid #c62828; margin-top:20px;'>
            <h4 style='margin-top:0; color:#b71c1c;'>⚠️ CRITICAL: Understanding Negative Drop to Margin Call</h4>
            <ul style='padding-left: 20px;'>
                <li><b style='color:#b71c1c;'>Negative Values (e.g., -18%):</b> This means the portfolio is <b>over-leveraged</b>. Your equity is currently below the {maintenance_margin:.0%} maintenance requirement. You would receive an <b>immediate margin call</b>.</li>
                <li><b>Net Monthly Cash:</b> Note the <b>-${abs(comp_df.loc['Desired CML Portfolio', 'Net Monthly Cash (usd)']):,.2f}</b> for the Desired CML. You are losing more in interest than you gain in dividends.</li>
                <li><b>Solution:</b> Use the Risk Slider to <b>reduce your Risk Budget</b>. Aim for a "Drop to Margin Call" value that is positive and provides a comfortable safety buffer (e.g., > 30%).</li>
            </ul>
            </div>"""))

            is_busy = False 

    # Link Widgets
    target_mode.observe(toggle_inputs, 'value')
    for w in [target_mode, target_val_type, target_input, risk_slider, compare_to, rf_input, margin_input, portfolio_value_input]:
        w.observe(update_dashboard, 'value')

    display(widgets.VBox([portfolio_value_input, widgets.HBox([rf_input, margin_input]), widgets.HBox([target_mode, target_val_type, target_input]), range_info, widgets.HBox([risk_slider, status_label]), compare_to, out]))
    update_dashboard(); toggle_inputs(None)



    

## Polted Solution

In [52]:
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def efficient_froentier_sharp_ratio(
        rets: pd.DataFrame, 
        curr_port: list, 
        mean_ann: pd.Series, 
        cov_ann: pd.DataFrame, 
        rf: float,
        long_only: bool,
        n_starts: int,
        n_sims: int,
        seed: int
        ):
    """
    Assets: A series of Stocks and or Indices
    Current or Initial Portfolio: The existing Portfolio we want to improve
    Creates random portfolios using the Assests provided in the 'rets' dataframe
    Imports Current or Initial Portfolio information to Compare against the calculated portfolios
    Calculates Sharp Ratio of different Portfolios
    Plots the Efficient Frontier
    Args:
        rets (pd.DataFrame): Dataframe that includes Daily Returns of all Assets to study
        curr_port: (list): [volatility, Return] List with two Dataframes: Volatility and Return of the Current or Initial Portfolio
        mean_ann (pandas Series): Vector of Mean of Returns (Annualized) of the Assets
        cov_ann (pd.DataFrame): Covariance Variance Matrix (Annualized) of the Assets
        rf (float): Risk Free Rate in decimal
        long_only (bool): Long only = True, Short allowed = False
        n_starts (int): Number of starts for SLSQP Solver, to avoid local minima
        n_sims (int): Number of simulations
        seed (int): Seed for randomnes and repeatidibility of simulations
    Returns:
        Weights of Optimal Portfolio
        Volatility, Return and Sharpe Ratio of the Optiomal Portfolio
        Sharp Ratio portfolio risk and return
        Plots Efficient Frontier
        Plots Capital Market Line and tangent portfolio (Max SR)
    """
    try:


        # 1. Optimized Portfolio (max SR) using current portfolio's mean and covariance

        def sharpe_ratio(weights, mean_ann, cov_ann, rf):
            w = np.asarray(weights)
            mu  = float(w @ mean_ann.values)
            vol = float(np.sqrt(w @ cov_ann.values @ w))
            return (mu - rf) / vol if vol > 0 else -np.inf

        def neg_sharpe(weights, mean_ann, cov_ann, rf):
            return -sharpe_ratio(weights, mean_ann, cov_ann, rf)

        def optimize_sharpe(
            rets: pd.DataFrame,
            rf: float,
            periods_per_year: int,
            long_only: bool,
            n_starts: int,
            seed: int
        ):
            """
            Maximiza Sharpe (SLSQP) con múltiples inicios aleatorios.
            - long_only=True => bounds (0,1); False => (-1,1)
            - n_starts: reinicios aleatorios para evitar óptimos locales.
            """
            rng = np.random.default_rng(seed)
            mean_ann
            cov_ann
            n = rets.shape[1]

            bounds = [(0.0, 1.0)] * n if long_only else [(-1.0, 1.0)] * n
            cons   = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0},) #eq: equality constraint

            # Inicios: igualitario + Dirichlet (long-only) o normales (short)
            inits = [np.ones(n) / n]
            if long_only:
                inits += [rng.dirichlet(np.ones(n)) for _ in range(max(0, n_starts - 1))]
            else:
                for _ in range(n_starts - 1):
                    w0 = rng.normal(0, 0.5, size=n)
                    w0 = w0 / np.sum(np.abs(w0))  # normalización L1
                    inits.append(w0)

            best_res, best_fun = None, np.inf
            for x0 in inits:
                res = minimize(
                    neg_sharpe, x0=x0,
                    args=(mean_ann, cov_ann, rf),
                    method='SLSQP', bounds=bounds, constraints=cons, #SLSQP: Sequential Least Squares Programming
                    options={'maxiter': 500, 'ftol': 1e-12, 'disp': False}
                )
                if res.success and res.fun < best_fun:
                    best_res, best_fun = res, res.fun

            if best_res is None:
                raise RuntimeError("No se encontró solución factible en la optimización.")

            w_opt = pd.Series(best_res.x, index=rets.columns)
            sr_opt = -best_fun
            mu_opt = float(w_opt.values @ mean_ann.values)
            vol_opt = float(np.sqrt(w_opt.values @ cov_ann.values @ w_opt.values))

            return {
                "weights": w_opt,
                "sharpe": sr_opt,
                "mu_annual": mu_opt,
                "vol_annual": vol_opt,
                "mean_ann": mean_ann,
                "cov_ann": cov_ann
            }

        print("\nOptimizing Sharpe…")
        result = optimize_sharpe(
            rets, rf=rf, periods_per_year=252,
            long_only=long_only, n_starts=n_starts, seed=seed
        )

        # Optimum Portfolio (Best Sharpe ratio)
        opt_volatility = result["vol_annual"]
        opt_return     = result["mu_annual"]
        sharpe_opt     = result["sharpe"]

        w_opt = (result["weights"] * 100).round(2)
        print("\nOptimum Weights (%) - Tangency Portfolio")
        print(w_opt.to_string())
        print(f"\nOptimum Sharpe Ratio: {result['sharpe']:.3f}")
        print(f"Expected Annual Return: {result['mu_annual']:.2%}")
        print(f"Annual Risk: {result['vol_annual']:.2%}")



        # 2. Simulated Portfolios using current portfolio's mean and covariance

        def simulate_random_portfolios(mean_ann: pd.Series, cov_ann: pd.DataFrame, n_sims: int, rf: float, long_only: bool, seed: int):
            """
            Generates random Portfolios and returns [risk, return, SR]
            """
            rng = np.random.default_rng(seed)
            m, C = mean_ann.values, cov_ann.values
            res = np.zeros((3, n_sims))
            d = len(m)
            for i in range(n_sims):
                if long_only:
                    w = rng.dirichlet(np.ones(d))
                else:
                    w = rng.normal(0, 0.5, size=d)
                    # normalizar a suma de pesos = 1 (permitiendo negativos)
                    s = np.sum(w)
                    w = w / s if s != 0 else w / np.sum(np.abs(w))
                mu  = float(w @ m)
                vol = float(np.sqrt(w @ C @ w))
                sr  = (mu - rf) / vol if vol > 0 else np.nan
                res[0, i] = vol
                res[1, i] = mu
                res[2, i] = sr

            return res
        
        def min_max_sim_port (sim): 
            """
                Args
                    sim: Simulated Portfolios [vol. ret, sr]
                Returns
                    min vol and max ret portolios
            """
            # Minimum Volatility (Risk) Simulated Portfolio
            min_risk_idx = np.argmin(sim[0,:]) #index
            min_risk_vol = sim[0,min_risk_idx]
            min_risk_ret = sim[1,min_risk_idx]
            min_risk_sr = sim[2,min_risk_idx]
            print(f"\nSimulated Portfolio with Minimum Volatility:")
            print(f"Volatility: {min_risk_vol:.2%}")
            print(f"Return: {min_risk_ret:.2%}")
            print(f"Sharpe Ratio: {min_risk_sr:.3f}")
            # Max Return Simulated Portfolio
            max_ret_idx = np.argmax(sim[1,:]) #index
            max_ret_vol = sim[0,max_ret_idx]
            max_ret_ret = sim[1,max_ret_idx]
            max_ret_sr = sim[2,max_ret_idx]
            print(f"\nSimulated Portfolio with Maximum Return:")
            print(f"Volatility: {max_ret_vol:.2%}")
            print(f"Return: {max_ret_ret:.2%}")
            print(f"Sharpe Ratio: {max_ret_sr:.3f}")

            return {
                "min_risk_vol": min_risk_vol,
                "min_risk_ret": min_risk_ret,
                "max_ret_vol": max_ret_vol,
                "max_ret_ret": max_ret_ret
            }

        # Simulación para graficar "frontera" aleatoria
        print("\nSimulating random Portfolios…")
        sim = simulate_random_portfolios(
            result["mean_ann"], result["cov_ann"],
            n_sims=n_sims, rf=rf, long_only=long_only, seed=seed
        )

        # Min_Volatility and Max_Return Portfolios (from simulations)
        min_max = min_max_sim_port(sim)



        # 3. Current Portfolio

        curr_port_volatility, curr_port_return = curr_port
        sharpe_ratio_curr_port = (curr_port_return - rf) / curr_port_volatility
        print(f"\nCurrent Portfolio:")
        print(f"Volatility: {curr_port_volatility:.2%}")
        print(f"Return: {curr_port_return:.2%}")
        print(f"Sharpe Ratio: {sharpe_ratio_curr_port:.3f}")




        # 4. Capital Market Line

        sigma_values = np.linspace(0, 0.5, 100) # Range of volatilities for the CML
        CML_line = rf + (sharpe_opt * sigma_values)  # Equation: E[R] = Rf + S_max * σ




        # 5. Plots

        plt.figure(figsize=(10, 6))
        sc = plt.scatter(sim[0, :], sim[1, :], c=sim[2, :], cmap='viridis',
                        alpha=0.35, edgecolors='none', label='Random Portfolios')
        plt.colorbar(sc, label='Sharpe Ratio')
        plt.scatter(opt_volatility, opt_return, color='red', marker='*', s=220, label='Max Sharpe Ratio (optimizer) or Tangency Portfolio')
        plt.annotate(f'({opt_volatility:.2%}, {opt_return:.2%}, sr:{sharpe_opt:.2f})', xy=(opt_volatility, opt_return), xytext=(opt_volatility+0.01, opt_return-0.01), arrowprops=dict(arrowstyle='->'))
        plt.scatter(curr_port_volatility, curr_port_return, color='blue', marker='*', s=220, label='Current Portfolio')        
        plt.scatter(min_max['min_risk_vol'], min_max['min_risk_ret'], color='green', marker='*', s=220, label='Min Volatility Simulated')        
        plt.scatter(min_max['max_ret_vol'], min_max['max_ret_ret'], color='black', marker='*', s=220, label='Max Return Simulated')        
        plt.scatter(0, rf, color='orange', marker='o', s=80, label=f'Risk-Free Asset {rf*100}%')                
        plt.plot(sigma_values, CML_line, linestyle='--', label=f"Capital Market Line (slope = {sharpe_opt:.3f})")
        plt.xlabel('Annualized Volatility')
        plt.ylabel('Expected Return  E[R]  Annualized')
        title_suffix = "Long-only" if long_only else "Short Allowed"
        plt.title(f'Efficient Frontier (simulated) vs Capital Market Line (CML) • {title_suffix}')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()

        weights_optimal = pd.DataFrame(w_opt/100, columns=['Optimal Weights'])
        vol_ret_sr_optimal = [opt_volatility, opt_return, sharpe_opt]        
        return weights_optimal, vol_ret_sr_optimal

    except (KeyError, IndexError):
        print(f"Error (efficient_froentier_sharp_ratio)")
        return None
    except Exception as e:
        print(f"(efficient_froentier_sharp_ratio) An unexpected error occurred: {e}")
        return None
    

## Expected Utility Function and the Indifference Curves:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_portfolio_utility(risk_free, tangency_ret, tangency_vol, gammas=[2, 4, 8]):
    """
    # INDIFFERENCE CURVES

    # This function shows how different risk aversion levels (𝛾) result in different "optimal" points where the curves touch the CAL.
    # The curves (indifference curves) represent levels of equal utility, while the straight line is the Capital Allocation Line (CAL).

    """
    # 1. Setup Data
    sharpe = (tangency_ret - risk_free) / tangency_vol
    max_y = (tangency_ret - risk_free) / (min(gammas) * (tangency_vol**2))
    max_vol = max_y * tangency_vol * 1.1
    vols = np.linspace(0, max_vol, 250)
    cal_returns = risk_free + sharpe * vols

    plt.figure(figsize=(14, 9))
    plt.plot(vols, cal_returns, color='black', linewidth=2, label='CAL (Capital Allocation Line)', zorder=1)

    # 2. Plot Curves and Points
    colors = ['green', 'blue', 'red']
    for i, g in enumerate(gammas):
        y_star = (tangency_ret - risk_free) / (g * (tangency_vol**2))
        opt_ret = risk_free + y_star * (tangency_ret - risk_free)
        opt_vol = y_star * tangency_vol
        u_max = opt_ret - 0.5 * g * (opt_vol**2)
        curve_rets = u_max + 0.5 * g * (vols**2)
        
        plt.plot(vols, curve_rets, '--', color=colors[i], alpha=0.4, label=fr'Utility ($\gamma$={g})')
        plt.scatter(opt_vol, opt_ret, color=colors[i], s=100, edgecolors='black', zorder=5)
        
        label_type = "Borrowing" if y_star > 1 else "Lending"
        # Added Risk (vol) and Return values to the annotation
        plt.annotate(fr'{label_type} ($\gamma$={g}) $y^*$={y_star:.1%}' + "\n" + fr'Risk: {opt_vol:.2%}, Ret: {opt_ret:.2%}', 
                     (opt_vol, opt_ret), xytext=(0, 15), textcoords='offset points', 
                     ha='center', fontweight='bold', bbox=dict(boxstyle='round,pad=0.3', fc='white', alpha=0.7, ec=colors[i]))

    # 3. Tangent Portfolio Marker
    plt.scatter(tangency_vol, tangency_ret, color='gold', s=250, marker='*', 
                label='Tangent Portfolio (Max Sharpe)', zorder=6, edgecolors='black')
    
    # Added Risk and Return values to Tangent Portfolio
    plt.annotate(fr'Tangent Portfolio' + "\n" + fr'Risk: {tangency_vol:.2%}, Ret: {tangency_ret:.2%}',
                 (tangency_vol, tangency_ret), xytext=(10, -25), textcoords='offset points',
                 ha='left', fontsize=10, fontweight='bold', color='darkgoldenrod')

    # 4. Title and Subtitle
    plt.suptitle("Portfolio Optimization: CAL vs. Indifference Curves", fontsize=18, fontweight='bold', y=0.98)
    plt.title("Capital Allocation Line (CAL): Borrowing vs. Lending across Risk Aversions", fontsize=14, pad=20)
    
    # --- ADDING THE NOTE BOX ---
    note_text = (fr"Note on y*:" + "\n"
                 fr"$y^*$ > 100%: Borrowing (Leverage)" + "\n"
                 fr"$y^*$ < 100%: Lending (Allocation to Cash)" + "\n"
                 fr"Formula: $y^*$ = (E[R_p] - R_f)/($\gamma$ $\sigma_p^2$)")
    
    plt.gca().text(0.65, 0.05, note_text, transform=plt.gca().transAxes, fontsize=10,
                   verticalalignment='bottom', horizontalalignment='left',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

    plt.xlabel(fr"Volatility ($\sigma_p$)", fontsize=12)
    plt.ylabel(fr"Expected Return ($E[R_p]$)", fontsize=12)
    plt.ylim(0, max(cal_returns) * 1.5)
    plt.xlim(0, max_vol)
    plt.grid(True, alpha=0.2)
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()


# Indicadores: alpha, Beta, R^2, SR, Sortino, VaR, CVaR

In [53]:
# @title
# !pip install yfinance scipy ipywidgets --quiet

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

from scipy.optimize import minimize
from scipy.stats import norm

import ipywidgets as w
from IPython.display import display, clear_output

import traceback


def indicators (
        tickers: list, 
        start_date: datetime, 
        today: datetime, 
        benchmark_indices: dict, 
        index_benchmark: int,
        risk_free: float,
        portfolio_weights: pd.Series,
        no_starts: int
        ):

    """
    Does: Computes Indicators alpha, Beta, R^2, SR, Sortino, VaR, CVaR
    Args: (see list)
    Returns: void

    """


    try:

        # Input data
        bench_ticker = list(benchmark_indices.values())[index_benchmark] 
        start = start_date.date() 
        end   = today.date() 
        rf    = risk_free
        kind = "simple" #"log" "simple"
        wmode = "opt" #("Igual","equal"),("Optimizar Sharpe","opt")
        var_level = 0.95 # ("90%",0.90),("95%",0.95),("99%",0.99)
        var_method= "historical" # "historical", "parametric"
        nstarts = no_starts
        seed    = 123 
        # ===================== Utilidades =====================

        def download_close(tickers, start, end, auto_adjust=True):
            df = yf.download(tickers, start=start, end=end, auto_adjust=True, progress=False)
            if isinstance(df.columns, pd.MultiIndex):
                close = df['Close'].copy()
            else:
                # Caso de un único ticker -> normalizamos a DataFrame con una columna
                close = df[['Close']].copy()
                colname = tickers if isinstance(tickers, str) else tickers[0]
                close.columns = [colname]
            close = close.dropna(how='all').dropna(axis=1, how='all')
            return close

        def download_bench(bench, start, end, auto_adjust=True):
            s = yf.download(bench, start=start, end=end, auto_adjust=True, progress=False)
            if isinstance(s.columns, pd.MultiIndex):
                s.columns = s.columns.get_level_values(0)
            s = s['Close'].copy()
            s = pd.Series(s, name=str(bench)).dropna()
            return s

        def compute_returns(prices: pd.DataFrame | pd.Series, kind="log"):
            if kind=="log":
                return np.log(prices/prices.shift(1)).dropna()
            return prices.pct_change(fill_method=None).dropna()

        def annualize_stats(rets: pd.DataFrame, periods=252):
            mu = rets.mean()*periods
            cov = rets.cov()*periods
            return mu, cov

        def optimize_sharpe(rets: pd.DataFrame, rf=risk_free, periods=252, n_starts=no_starts, long_only=True, seed=123):
            rng = np.random.default_rng(seed)
            mu_ann, cov_ann = annualize_stats(rets, periods)
            n = rets.shape[1]
            bounds = [(0,1)]*n if long_only else [(-1,1)]*n
            cons = ({'type':'eq','fun':lambda w: np.sum(w)-1},)
            inits = [np.ones(n)/n]
            if long_only:
                inits += [rng.dirichlet(np.ones(n)) for _ in range(max(0, n_starts-1))]
            else:
                for _ in range(n_starts-1):
                    w0 = rng.normal(0,0.5,n)
                    w0 = w0/np.sum(np.abs(w0))
                    inits.append(w0)
            def neg_sharpe(w):
                mu = float(w@mu_ann.values)
                vol = float(np.sqrt(w@cov_ann.values@w))
                return -((mu-rf)/vol if vol>0 else -1e9)
            best=None; bestf=np.inf
            for x0 in inits:
                res=minimize(neg_sharpe, x0, method='SLSQP', bounds=bounds, constraints=cons,
                            options={'maxiter':500,'ftol':1e-12})
                if res.success and res.fun<bestf:
                    best, bestf = res, res.fun
            if best is None: raise RuntimeError("No se encontró solución factible.")
            w_opt = pd.Series(best.x, index=rets.columns)
            mu_p = float(w_opt.values @ mu_ann.values)
            vol_p = float(np.sqrt(w_opt.values @ cov_ann.values @ w_opt.values))
            sr = (mu_p - rf)/vol_p if vol_p>0 else np.nan
            return {"weights": w_opt, "mu_ann": mu_p, "vol_ann": vol_p, "sharpe": sr,
                    "mu_vec": mu_ann, "cov": cov_ann}

        def jensen_alpha_beta(port_ret, bench_ret, rf_annual=risk_free, periods=252):
            """Alpha (Jensen) y Beta (CAPM) por regresión de excesos diarios, alpha anualizada."""
            rf_d = rf_annual/periods
            x = (bench_ret - rf_d).dropna()
            y = (port_ret  - rf_d).dropna()
            df = pd.concat([x,y], axis=1).dropna()
            if df.empty: return np.nan, np.nan, np.nan
            xb = df.iloc[:,0].values
            yb = df.iloc[:,1].values
            # Regresión OLS con intercepto: y = a + b x
            b, a = np.polyfit(xb, yb, 1)
            # alpha diario = a; beta = b
            alpha_ann = a * periods
            # R^2
            r = np.corrcoef(xb, yb)[0,1] if len(df)>1 else np.nan #Pearson Correlation coefficient
            r2 = r**2 if pd.notna(r) else np.nan
            return alpha_ann, b, r2

        def sharpe_ratio_series(ret, rf_annual=risk_free, periods=252):
            mu_ann = ret.mean()*periods
            vol_ann = ret.std(ddof=1)*np.sqrt(periods)
            rf = rf_annual
            return (mu_ann - rf)/vol_ann if vol_ann>0 else np.nan

        def sortino_ratio_series(ret, rf_annual=risk_free, periods=252):
            rf_d = rf_annual/periods
            ex = ret - rf_d #excess returns
            downside = np.minimum(ex, 0.0) #Isolate downside deviations 
            dd = downside.std(ddof=1) * np.sqrt(periods) #downside deviation (annualized)
            mu_ex_ann = ex.mean()*periods # Calculate annualized excess mean return
            return (mu_ex_ann)/dd if dd>0 else np.nan

        def var_cvar(ret, level=0.95, method="historical", rf_annual=risk_free, periods=252):
            """VaR/CVaR 1-día (no anualiza). Ret es serie diaria."""
            r = ret.dropna().values
            if r.size==0: return np.nan, np.nan
            alpha = level
            if method=="historical":
                q = np.quantile(r, 1-alpha)
                var_d = -q
                cvar_d = -r[r<=q].mean() if (r<=q).any() else np.nan
                return var_d, cvar_d
            # paramétrico (normal)
            mu_d, sd_d = np.mean(r), np.std(r, ddof=1)
            z = norm.ppf(1-alpha)
            var_d = -(mu_d + sd_d*z)
            cvar_d = -(mu_d - sd_d * (norm.pdf(z)/(1-alpha)))
            return var_d, cvar_d

        def cum_return(ret):
            return (1+ret).cumprod()
        




        clear_output()

        print("Descargando datos…")
        prices = download_close(tickers, start, end, auto_adjust=True)
        bench_close = download_bench(bench_ticker, start, end, auto_adjust=True)

        # Daily Returns
        rets = compute_returns(prices, kind) # Daily Returns of the Stocks of the Current Portfolio
        bench_ret = compute_returns(bench_close, kind)

        # Alinear fechas
        idx = rets.index.intersection(bench_ret.index)
        rets = rets.loc[idx]
        bench_ret = bench_ret.loc[idx]
        
        # Summary Benchmark
        mu_b = bench_ret.mean()*252
        vol_b = bench_ret.std(ddof=1)*np.sqrt(252)
        sharpe_b = (mu_b - rf)/(vol_b if vol_b>0 else np.nan)
            
        # Weights of Optimized Portfolio or for Equal Portfolio
        if wmode == "equal":
            w_ = pd.Series(np.ones(rets.shape[1])/rets.shape[1], index=rets.columns)
            mu_ann, cov_ann = annualize_stats(rets)
            mu_p = float(w_.values @ mu_ann.values)
            vol_p = float(np.sqrt(w_.values @ cov_ann.values @ w_.values))
            sharpe_p = (mu_p - rf)/vol_p if vol_p>0 else np.nan
        else:
            res = optimize_sharpe(rets, rf=rf, periods=252, n_starts=nstarts, long_only=True, seed=seed)
            w_ = res["weights"]
            mu_p, vol_p, sharpe_p = res["mu_ann"], res["vol_ann"], res["sharpe"]
        
        # Weights Current portfolio
        w_cp = portfolio_weights


        # Retorno del portafolio optimizado (annualized)
        port_ret = (rets @ w_.values)
        # Retorno del Current Portfolio (Daily Returns)
        port_curr_ret = (rets @ w_cp)

        # Métricas principales del Portafolio Optimizado (or could be the equal weights)
        sharpe_val = sharpe_ratio_series(port_ret, rf_annual=rf, periods=252)
        sortino_val = sortino_ratio_series(port_ret, rf_annual=rf, periods=252)
        alpha_ann, beta, r2 = jensen_alpha_beta(port_ret, bench_ret, rf_annual=rf, periods=252)
        # VaR / CVaR 1-día
        var_d, cvar_d = var_cvar(port_ret, level=var_level, method=var_method)


        # Métricas principales del Current Portfolio
        sharpe_val_cp = sharpe_ratio_series(port_curr_ret, rf_annual=rf, periods=252)
        sortino_val_cp = sortino_ratio_series(port_curr_ret, rf_annual=rf, periods=252)
        alpha_ann_cp, beta_cp, r2_cp = jensen_alpha_beta(port_curr_ret, bench_ret, rf_annual=rf, periods=252)
        # VaR / CVaR 1-día
        var_d_cp, cvar_d_cp = var_cvar(port_curr_ret, level=var_level, method=var_method)

        # Summary Current Portfolio
        mu_cp = port_curr_ret.mean()*252
        vol_cp = port_curr_ret.std(ddof=1)*np.sqrt(252)

        # Tabla de resultados

        summary = pd.DataFrame({
            "μ anual": [mu_cp, mu_p, mu_b],
            "σ anual": [vol_cp, vol_p, vol_b],
            "Sharpe": [sharpe_val_cp, sharpe_val, sharpe_b],
            "Sortino": [sortino_val_cp, sortino_val, np.nan],
        }, index=["Current Portfolio", "Portafolio (max SR)",f"Benchmark [{bench_ticker}]"]).round(4)

        capm = pd.DataFrame({
            "Alpha (anual)": [alpha_ann_cp, alpha_ann],
            "Beta": [beta_cp, beta],
            "R²": [r2_cp, r2],
            "VaR 1d": [var_d_cp, var_d],
            "CVaR 1d": [cvar_d_cp, cvar_d],
        }, index=["Current Portfolio", "Portafolio (max SR)"]).round(4)

        print("\nWeights: ")
        w_df = pd.DataFrame(w_, columns=['Optimized Portfolio (max SR) Weights'])
        portfolio_weights.name = "Current Portfolio Weights"
        weights = pd.concat([w_df, portfolio_weights], axis=1)
        print(round(weights,2))

        print("\nMétricas (anualizadas)")
        display(summary)
        print(f"Exceso risk-free usado: {rf:.2%} anual | Rend: {kind} | VaR: {int(var_level*100)}% ({'historical' if var_method=='historical' else 'parametric'})")

        print(f"\nAlpha/Beta/R²/VaR/CVaR con respecto a {bench_ticker}")
        display(capm)

        # Gráfico acumulado
        nav_cp = cum_return(port_curr_ret) # Current portfolio
        nav_p = cum_return(port_ret) #max SR portfolio
        nav_b = cum_return(bench_ret) # benchmark
        plt.figure(figsize=(10,5))
        plt.plot(nav_p.index, nav_p.values, label="Portafolio (max SR)")
        plt.plot(nav_cp.index, nav_cp.values, label=f"Current Portfolio)", alpha=0.8)
        plt.plot(nav_b.index, nav_b.values, label=f"Benchmark ({bench_ticker})", alpha=0.8)
        plt.title("Retorno Acumulado (base=1.0 o Normalizado)")
        plt.ylabel("Crecimiento")
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.show()

        return
    
    except (KeyError, IndexError):
        print(f"Error (indicators)")
        return None
    except Exception as e:
        print(f"(indicators) An unexpected error occurred: {e}")
        print(traceback.format_exc())
        return None
